In [ ]:
import pandas as pd

# Load main tables
patients = pd.read_csv("PATIENTS.csv")
admissions = pd.read_csv("ADMISSIONS.csv")
icustays = pd.read_csv("ICUSTAYS.csv")
diagnoses = pd.read_csv("DIAGNOSES_ICD.csv")
diag_lookup = pd.read_csv("D_ICD_DIAGNOSES.csv")
procedures = pd.read_csv("PROCEDURES_ICD.csv")
proc_lookup = pd.read_csv("D_ICD_PROCEDURES.csv")
prescriptions = pd.read_csv("PRESCRIPTIONS.csv")
labs = pd.read_csv("LABEVENTS.csv")
lab_lookup = pd.read_csv("D_LABITEMS.csv")
microbio = pd.read_csv("MICROBIOLOGYEVENTS.csv")
inputevents = pd.read_csv("INPUTEVENTS_CV.csv")
outputevents = pd.read_csv("OUTPUTEVENTS.csv")
drgcodes = pd.read_csv("DRGCODES.csv")
transfers = pd.read_csv("TRANSFERS.csv")

/tmp/ipython-input-211632253.py:15: DtypeWarning: Columns (17,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  inputevents = pd.read_csv("INPUTEVENTS_CV.csv")


In [ ]:
patients_clean = patients[["subject_id", "gender", "dob", "dod", "expire_flag"]]
admissions_clean = admissions[[
    "subject_id", "hadm_id", "admittime", "dischtime", "deathtime",
    "admission_type", "ethnicity", "diagnosis",
    "insurance", "marital_status", "religion"
]]
icustays_clean = icustays[[
    "subject_id", "hadm_id", "icustay_id", "intime", "outtime", "los", "first_careunit"
]]

adm_demo = admissions_clean.merge(patients_clean, on="subject_id", how="left")
adm_demo = adm_demo.merge(icustays_clean, on=["subject_id", "hadm_id"], how="left")

diagnoses = diagnoses.merge(diag_lookup, on="icd9_code", how="left")
diagnoses_clean = diagnoses[["subject_id", "hadm_id", "icd9_code", "long_title", "seq_num"]]

procedures = procedures.merge(proc_lookup, on="icd9_code", how="left")
procedures_clean = procedures[["subject_id", "hadm_id", "icd9_code", "long_title", "seq_num"]]

prescriptions_clean = prescriptions[[
    "subject_id", "hadm_id", "drug", "startdate", "enddate", "dose_val_rx", "dose_unit_rx", "route"
]].dropna(subset=["drug"])

labs = labs.merge(lab_lookup, on="itemid", how="left")
labs_clean = labs[[
    "subject_id", "hadm_id", "charttime", "label", "value", "flag"
]].dropna(subset=["label"])

microbio_clean = microbio[[
    "subject_id", "hadm_id", "chartdate", "spec_type_desc", "org_name", "ab_name", "interpretation"
]].dropna(subset=["spec_type_desc", "org_name"])

inputevents_clean = inputevents[[
    "subject_id", "hadm_id", "charttime", "itemid", "amount", "rate"
]].dropna(subset=["amount"])

outputevents_clean = outputevents[[
    "subject_id", "hadm_id", "charttime", "itemid", "value"
]].dropna(subset=["value"])

transfers_clean = transfers[[
    "subject_id", "hadm_id", "icustay_id", "eventtype", "prev_careunit", "curr_careunit", "intime", "outtime"
]]

drgcodes_clean = drgcodes[[
    "subject_id", "hadm_id", "drg_code", "description"
]]

In [ ]:
def get_patient_report(subject_id_val):
    data = adm_demo[adm_demo["subject_id"] == subject_id_val]

    dx = diagnoses_clean[diagnoses_clean["subject_id"] == subject_id_val]
    px = procedures_clean[procedures_clean["subject_id"] == subject_id_val]
    rx = prescriptions_clean[prescriptions_clean["subject_id"] == subject_id_val]
    lb = labs_clean[labs_clean["subject_id"] == subject_id_val]
    mb = microbio_clean[microbio_clean["subject_id"] == subject_id_val]
    iv = inputevents_clean[inputevents_clean["subject_id"] == subject_id_val]
    ov = outputevents_clean[outputevents_clean["subject_id"] == subject_id_val]
    tr = transfers_clean[transfers_clean["subject_id"] == subject_id_val]
    drg = drgcodes_clean[drgcodes_clean["subject_id"] == subject_id_val]

    return {
        "admission_info": data,
        "diagnoses": dx,
        "procedures": px,
        "prescriptions": rx,
        "labs": lb,
        "microbiology": mb,
        "inputs": iv,
        "outputs": ov,
        "transfers": tr,
        "drg_codes": drg
    }

In [ ]:
get_patient_report(10006)

{'admission_info':    subject_id  hadm_id            admittime            dischtime deathtime  \
 0       10006   142345  2164-10-23 21:09:00  2164-11-01 17:15:00       NaN   
 
   admission_type               ethnicity diagnosis insurance marital_status  \
 0      EMERGENCY  BLACK/AFRICAN AMERICAN    SEPSIS  Medicare      SEPARATED   
 
    religion gender                  dob                  dod  expire_flag  \
 0  CATHOLIC      F  2094-03-05 00:00:00  2165-08-12 00:00:00            1   
 
    icustay_id               intime              outtime     los first_careunit  
 0      206504  2164-10-23 21:10:15  2164-10-25 12:21:07  1.6325           MICU  ,
 'diagnoses':     subject_id  hadm_id icd9_code  \
 0        10006   142345     99591   
 1        10006   142345     99662   
 2        10006   142345      5672   
 3        10006   142345     40391   
 4        10006   142345     42731   
 5        10006   142345      4280   
 6        10006   142345      4241   
 7        10006   14

In [ ]:
def generate_text_summary_enhanced(report):
    summary = ""

    # 1. Demographics & Admission
    adm = report["admission_info"]
    if not adm.empty:
        row = adm.iloc[0]
        summary += f"Patient is a {row['gender']} admitted on {row['admittime'].split()[0]} for {row['admission_type'].lower()} care with a primary diagnosis of '{row['diagnosis']}'. "
        summary += f"Ethnicity: {row['ethnicity']}, Marital Status: {row['marital_status']}, Insurance: {row['insurance']}. "
        summary += f"Religion: {row['religion']}. Length of ICU stay: {row.get('los', 'N/A')} days.\n\n"
    else:
        summary += "No admission data available.\n\n"

    # 2. Diagnoses
    dx = report["diagnoses"]
    if not dx.empty:
        diagnoses = dx.sort_values("seq_num")["long_title"].dropna().unique().tolist()
        summary += f"Diagnoses include: {', '.join(diagnoses)}.\n\n"
    else:
        summary += "No diagnosis records found.\n\n"

    # 3. Procedures
    px = report["procedures"]
    if not px.empty:
        procedures = px.sort_values("seq_num")["long_title"].dropna().unique().tolist()
        summary += f"Procedures performed: {', '.join(procedures)}.\n\n"
    else:
        summary += "No procedures recorded.\n\n"

    # 4. Medications
    rx = report["prescriptions"]
    if not rx.empty:
        drugs = rx["drug"].dropna().unique().tolist()
        summary += f"Medications prescribed: {', '.join(drugs[:10])}.\n\n"
    else:
        summary += "No prescriptions data available.\n\n"

    # 5. Labs (abnormal only)
    lb = report["labs"]
    if not lb.empty:
        abnormal = lb[lb["flag"] == "abnormal"]
        abnormal_labs = abnormal["label"].dropna().unique().tolist()
        summary += f"Abnormal lab results observed in: {', '.join(abnormal_labs[:10])}.\n\n"
    else:
        summary += "No lab results available.\n\n"

    # 6. Microbiology
    mb = report["microbiology"]
    if not mb.empty:
        mb = mb.dropna(subset=["spec_type_desc", "org_name"])
        orgs = mb["org_name"].dropna().unique().tolist()
        summary += f"Microbiology cultures detected organisms: {', '.join(orgs[:5])}.\n\n"
    else:
        summary += "No microbiology records available.\n\n"

    # 7. Input Events
    iv = report["inputs"]
    if not iv.empty:
        avg_iv = iv["amount"].dropna().mean()
        summary += f"Average IV fluid/medication input: {round(avg_iv, 2)} units.\n\n"
    else:
        summary += "No input event records available.\n\n"

    # 8. Output Events
    ov = report["outputs"]
    if not ov.empty:
        avg_ov = ov["value"].dropna().mean()
        summary += f"Average fluid output (e.g. urine): {round(avg_ov, 2)} units.\n\n"
    else:
        summary += "No output event records available.\n\n"

    # 9. Transfers
    tr = report["transfers"]
    if not tr.empty:
        careunits = tr["curr_careunit"].dropna().unique().tolist()
        summary += f"Patient transferred through units: {', '.join(careunits)}.\n\n"
    else:
        summary += "No transfer data available.\n\n"

    # 10. DRG Codes
    drg = report["drg_codes"]
    if not drg.empty:
        codes = drg["description"].dropna().unique().tolist()
        summary += f"DRG complexity descriptions: {', '.join(codes[:5])}.\n\n"
    else:
        summary += "No DRG code data found.\n\n"

    return summary

In [ ]:
patient_data = get_patient_report(10006)
text_summary = generate_text_summary_enhanced(patient_data)
print(text_summary)

Patient is a F admitted on 2164-10-23 for emergency care with a primary diagnosis of 'SEPSIS'. Ethnicity: BLACK/AFRICAN AMERICAN, Marital Status: SEPARATED, Insurance: Medicare. Religion: CATHOLIC. Length of ICU stay: 1.6325 days.

Diagnoses include: Sepsis, Infection and inflammatory reaction due to other vascular device, implant, and graft, Hypertensive chronic kidney disease, unspecified, with chronic kidney disease stage V or end stage renal disease, Atrial fibrillation, Congestive heart failure, unspecified, Aortic valve disorders, Mitral valve disorders, Other staphylococcal septicemia, Tachycardia, unspecified, Kidney dialysis as the cause of abnormal reaction of patient, or of later complication, without mention of misadventure at time of procedure, Infection with microorganisms resistant to penicillins, Diverticulitis of colon (without mention of hemorrhage), Anemia of other chronic disease, Diabetes mellitus without mention of complication, type II or unspecified type, not st

In [ ]:
##pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [18]:
from transformers import pipeline
import pandas as pd
from tqdm import tqdm

# Load open-source LLM (Flan-T5 base for now; can swap with better model)
generator = pipeline("text2text-generation", model="google/flan-t5-base")

# Instruction template
def create_instruction(summary, target):
    if target == "family":
        return f"Explain the following medical case in simple, compassionate terms for the patient's family:\n\n{summary}"
    elif target == "doctor":
        return f"Summarize this patient's clinical case in medical language for a doctor:\n\n{summary}"
    elif target == "admin":
        return f"Summarize the key administrative and treatment aspects of this patient case:\n\n{summary}"
    else:
        return f"Summarize this case:\n\n{summary}"

# Dataset generation
def generate_finetuning_dataset(reports_dict, targets=["family", "doctor", "admin"], max_samples=None):
    dataset = []

    for i, (subject_id, report) in tqdm(enumerate(reports_dict.items()), total=max_samples or len(reports_dict), desc="Generating fine-tuning dataset"):
        if max_samples and i >= max_samples:
            break

        try:
            base_summary = generate_text_summary_enhanced(report).strip()

            for target in targets:
                instruction = create_instruction(base_summary, target)
                response = generator(instruction, max_length=512, truncation=True)[0]['generated_text']

                dataset.append({
                    "subject_id": subject_id,
                    "target_audience": target,
                    "input": base_summary,
                    "output": response.strip()
                })

        except Exception as e:
            print(f"⚠️ Error for subject {subject_id}: {e}")

    df = pd.DataFrame(dataset)
    df.to_csv("fine_tune_multi_target_dataset.csv", index=False)
    print(f"\n✅ Dataset saved: fine_tune_multi_target_dataset.csv with {len(df)} rows")
    return df

Device set to use cpu


In [19]:
# List of subject_ids you want to generate reports
subject_ids = adm_demo["subject_id"].dropna().unique().tolist()[:2]

# Build reports_dict
reports_dict = {}

for sid in subject_ids:
    report = get_patient_report(sid)
    reports_dict[sid] = report

In [20]:
reports_dict

{10006: {'admission_info':    subject_id  hadm_id            admittime            dischtime deathtime  \
  0       10006   142345  2164-10-23 21:09:00  2164-11-01 17:15:00       NaN   
  
    admission_type               ethnicity diagnosis insurance marital_status  \
  0      EMERGENCY  BLACK/AFRICAN AMERICAN    SEPSIS  Medicare      SEPARATED   
  
     religion gender                  dob                  dod  expire_flag  \
  0  CATHOLIC      F  2094-03-05 00:00:00  2165-08-12 00:00:00            1   
  
     icustay_id               intime              outtime     los first_careunit  
  0      206504  2164-10-23 21:10:15  2164-10-25 12:21:07  1.6325           MICU  ,
  'diagnoses':     subject_id  hadm_id icd9_code  \
  0        10006   142345     99591   
  1        10006   142345     99662   
  2        10006   142345      5672   
  3        10006   142345     40391   
  4        10006   142345     42731   
  5        10006   142345      4280   
  6        10006   142345      42

In [24]:
# Step 1: Choose number of patients and targets
num_samples = 2
target_audiences = ["family"]  # You can customize this

# Step 2: Generate fine-tuning dataset
final_df = generate_finetuning_dataset(
    reports_dict=reports_dict,
    targets=target_audiences,
    max_samples=2
)

# Step 3: View a few samples
print(final_df.head())


Generating fine-tuning dataset: 100%|██████████| 2/2 [01:55<00:00, 57.62s/it]


✅ Dataset saved: fine_tune_multi_target_dataset.csv with 2 rows
   subject_id target_audience  \
0       10006          family   
1       10011          family   

                                               input  \
0  Patient is a F admitted on 2164-10-23 for emer...   
1  Patient is a F admitted on 2126-08-14 for emer...   

                                              output  
0  The patient is a black/American American who h...  
1  Acute and subacute necrosis of liver, Viral he...  


In [25]:
final_df

,subject_id,target_audience,input,output
0,10006,family,Patient is a F admitted on 2164-10-23 for emer...,The patient is a black/American American who h...
1,10011,family,Patient is a F admitted on 2126-08-14 for emer...,"Acute and subacute necrosis of liver, Viral he..."


In [55]:
patient_data = get_patient_report(10033)
text_summary = generate_text_summary_enhanced(patient_data)
print(text_summary)

Patient is a F admitted on 2132-12-05 for emergency care with a primary diagnosis of 'RENAL FAILIURE-SYNCOPE-HYPERKALEMIA'. Ethnicity: WHITE, Marital Status: MARRIED, Insurance: Medicare. Religion: CATHOLIC. Length of ICU stay: 1.0695 days.

Diagnoses include: Toxic multinodular goiter without mention of thyrotoxic crisis or storm, Other diseases of trachea and bronchus, Acute kidney failure, unspecified, Hyperpotassemia, Atrial fibrillation, Unspecified fall, Unspecified essential hypertension, Diabetes mellitus without mention of complication, type II or unspecified type, not stated as uncontrolled, Pure hypercholesterolemia, Depressive disorder, not elsewhere classified, Late effects of cerebrovascular disease, aphasia, Peptic ulcer of unspecified site, unspecified as acute or chronic, without mention of hemorrhage or perforation, without mention of obstruction.

No procedures recorded.

Medications prescribed: Sodium Chloride 0.9%  Flush, Sodium Polystyrene Sulfonate, Metoprolol, R

In [29]:
# List of subject_ids you want to generate reports
subject_ids = adm_demo["subject_id"].dropna().unique().tolist()
print(subject_ids)

[10006, 10011, 10013, 10017, 10019, 10026, 10027, 10029, 10032, 10033, 10035, 10036, 10038, 10040, 10042, 10043, 10044, 10045, 10046, 10056, 10059, 10061, 10064, 10065, 10067, 10069, 10074, 10076, 10083, 10088, 10089, 10090, 10093, 10094, 10098, 10101, 10102, 10104, 10106, 10111, 10112, 10114, 10117, 10119, 10120, 10124, 10126, 10127, 10130, 10132, 40124, 40177, 40204, 40277, 40286, 40304, 40310, 40456, 40503, 40595, 40601, 40612, 40655, 40687, 41795, 41914, 41976, 41983, 42033, 42066, 42075, 42135, 42199, 42231, 42275, 42281, 42292, 42302, 42321, 42346, 42367, 42412, 42430, 42458, 43735, 43746, 43748, 43779, 43798, 43827, 43870, 43879, 43881, 43909, 43927, 44083, 44154, 44212, 44222, 44228]


In [34]:
# Example list to store the data
manual_dataset = []

# Function to add a single data row
def add_manual_entry(subject_id, target_audience, input_summary, output_summary):
    manual_dataset.append({
        "subject_id": subject_id,
        "target_audience": target_audience,
        "input": input_summary.strip(),
        "output": output_summary.strip()
    })

# Example usage:
add_manual_entry(
    subject_id=10006,
    target_audience="standard",
    input_summary="""Patient is a F admitted on 2164-10-23 for emergency care with a primary diagnosis of 'SEPSIS'. Ethnicity: BLACK/AFRICAN AMERICAN, Marital Status: SEPARATED, Insurance: Medicare. Religion: CATHOLIC. Length of ICU stay: 1.6325 days.

Diagnoses include: Sepsis, Infection and inflammatory reaction due to other vascular device, implant, and graft, Hypertensive chronic kidney disease, unspecified, with chronic kidney disease stage V or end stage renal disease, Atrial fibrillation, Congestive heart failure, unspecified, Aortic valve disorders, Mitral valve disorders, Other staphylococcal septicemia, Tachycardia, unspecified, Kidney dialysis as the cause of abnormal reaction of patient, or of later complication, without mention of misadventure at time of procedure, Infection with microorganisms resistant to penicillins, Diverticulitis of colon (without mention of hemorrhage), Anemia of other chronic disease, Diabetes mellitus without mention of complication, type II or unspecified type, not stated as uncontrolled, Long-term (current) use of insulin, Anticoagulants causing adverse effects in therapeutic use, Coronary atherosclerosis of native coronary artery, Gout, unspecified, Tobacco use disorder.

Procedures performed: Removal of other device from thorax, Percutaneous abdominal drainage, Venous catheterization for renal dialysis, Hemodialysis, Venous catheterization, not elsewhere classified, Transfusion of other serum, Injection or infusion of oxazolidinone class of antibiotics.

Medications prescribed: Sodium Chloride 0.9%  Flush, Glipizide, Metoprolol, Sevelamer, Insulin, Pantoprazole, Acetaminophen, Magnesium Oxide, Docusate Sodium, Senna.

Abnormal lab results observed in: Creatinine, Glucose, Phosphate, Urea Nitrogen, Vancomycin, Hematocrit, Hemoglobin, Lymphocytes, Neutrophils, PT.

Microbiology cultures detected organisms: STAPHYLOCOCCUS, COAGULASE NEGATIVE.

Average IV fluid/medication input: 315.0 units.

Average fluid output (e.g. urine): 55.0 units.

Patient transferred through units: MICU.

DRG complexity descriptions: SEPTICEMIA AGE >17.""",
    output_summary="""Female patient admitted on 2164-10-23 with a primary diagnosis of sepsis. Comorbidities included hypertensive chronic kidney disease (Stage V/ESRD), atrial fibrillation, congestive heart failure, valvular heart disease, type 2 diabetes mellitus, anemia of chronic disease, and coronary atherosclerosis. Patient also presented with infections related to vascular access and resistant microorganisms.

Procedures performed included removal of thoracic device, abdominal drainage, venous catheterization for hemodialysis, and transfusion therapy. Medications administered included broad-spectrum antibiotics (including oxazolidinones), insulin, cardiovascular agents, and supportive therapies.

Lab findings were notable for abnormalities in renal function (elevated creatinine and urea), electrolyte imbalance, anemia, coagulopathy, and infection markers. Microbiology cultures were positive for coagulase-negative staphylococcus.

The patient was admitted to the MICU and managed with IV fluids, dialysis, and close monitoring. ICU stay lasted approximately 1.63 days. Diagnosis-Related Group (DRG): Septicemia, Age >17."""
)


In [35]:
add_manual_entry(
    subject_id=10011,
    target_audience="standard",
    input_summary="""Patient is a F admitted on 2126-08-14 for emergency care with a primary diagnosis of 'HEPATITIS B'. Ethnicity: UNKNOWN/NOT SPECIFIED, Marital Status: SINGLE, Insurance: Private. Religion: CATHOLIC. Length of ICU stay: 13.8507 days.

Diagnoses include: Acute and subacute necrosis of liver, Viral hepatitis B without mention of hepatic coma, acute or unspecified, without mention of hepatitis delta, Chronic hepatitis C without mention of hepatic coma, Opioid type dependence, continuous, Thrombocytopenia, unspecified, Hyperosmolality and/or hypernatremia.

Procedures performed: Parenteral infusion of concentrated nutritional substances, Venous catheterization, not elsewhere classified.

No prescriptions data available.

Abnormal lab results observed in: Alanine Aminotransferase (ALT), Albumin, Alkaline Phosphatase, Ammonia, Asparate Aminotransferase (AST), Bilirubin, Direct, Bilirubin, Total, Lactate Dehydrogenase (LD), Potassium, Urea Nitrogen.

Microbiology cultures detected organisms: YEAST.

Average IV fluid/medication input: 95.97 units.

Average fluid output (e.g. urine): 181.22 units.

Patient transferred through units: MICU.

DRG complexity descriptions: DISORDERS OF LIVER EXCEPT MALIGNANCY, CIRRHOSIS, ALCOHOLIC HEPATITIS WITH COMPLICATIONS, COMORBIDITIES.""",
    output_summary="""Female patient admitted on 2126-08-14 for emergency care with a primary diagnosis of Hepatitis B. Notable comorbidities included acute and subacute hepatic necrosis, chronic hepatitis C, thrombocytopenia, opioid dependence, and hypernatremia.

The patient underwent venous catheterization and received parenteral nutrition support. No medication records were available for this admission.

Laboratory evaluation showed significant hepatic dysfunction (elevated ALT, AST, bilirubin, and LDH), hypoalbuminemia, hyperammonemia, and electrolyte disturbances. Microbiology cultures detected the presence of yeast organisms.

The patient was managed in the MICU for approximately 13.85 days, with an average fluid input of 95.97 units and output of 181.22 units. DRG classification indicated liver disorders with complications and comorbidities."""
)


In [37]:
add_manual_entry(
    subject_id=10013,
    target_audience="standard",
    input_summary="""Patient is a F admitted on 2125-10-04 for emergency care with a primary diagnosis of 'SEPSIS'. Ethnicity: UNKNOWN/NOT SPECIFIED, Marital Status: nan, Insurance: Medicare. Religion: CATHOLIC. Length of ICU stay: 2.6499 days.

Diagnoses include: Unspecified septicemia, Subendocardial infarction, initial episode of care, Cardiogenic shock, Pneumonia, organism unspecified, Atrial fibrillation, Other malignant lymphomas, unspecified site, extranodal and solid organ sites, Mitral valve disorders, Other and unspecified hyperlipidemia.

Procedures performed: Arterial catheterization.

Medications prescribed: Furosemide, Azithromycin, Acetaminophen, Nitroglycerin SL, Aspirin, Alteplase (Catheter Clearance), Magnesium Sulfate, Ipratropium Bromide Neb, Morphine Sulfate, Heparin Sodium.

Abnormal lab results observed in: Asparate Aminotransferase (AST), Chloride, CK-MB Index, Creatine Kinase (CK), Creatine Kinase, MB Isoenzyme, Creatinine, Glucose, Magnesium, Urea Nitrogen, Bands.

No microbiology records available.

Average IV fluid/medication input: 145.33 units.

Average fluid output (e.g. urine): 49.22 units.

Patient transferred through units: MICU.

DRG complexity descriptions: SEPTICEMIA AGE >17.""",
    output_summary="""Female patient admitted on 2125-10-04 for emergency care with a primary diagnosis of sepsis. Significant comorbidities included subendocardial myocardial infarction, cardiogenic shock, pneumonia, atrial fibrillation, malignant lymphoma, mitral valve disorder, and hyperlipidemia.

The patient underwent arterial catheterization and was administered medications including antibiotics (Azithromycin), diuretics (Furosemide), analgesics (Acetaminophen, Morphine), anticoagulants (Heparin), thrombolytics (Alteplase), and supportive agents (Nitroglycerin, Magnesium Sulfate).

Laboratory tests revealed abnormalities in cardiac enzymes (CK, CK-MB, CK-MB Index), renal markers (Creatinine, Urea Nitrogen), glucose, electrolytes (Magnesium, Chloride), and inflammatory bands.

No microbiology records were available. The patient had a 2.65-day stay in the MICU with average IV input of 145.33 units and output of 49.22 units. DRG classification: Septicemia, age >17."""
)


In [39]:
add_manual_entry(
    subject_id=10017,
    target_audience="standard",
    input_summary="""Patient is a F admitted on 2149-05-26 for emergency care with a primary diagnosis of 'HUMERAL FRACTURE'. Ethnicity: WHITE, Marital Status: DIVORCED, Insurance: Medicare. Religion: CATHOLIC. Length of ICU stay: 2.1436 days.

Diagnoses include: Closed fracture of surgical neck of humerus, Other emphysema, Closed fracture of other facial bones, Closed fracture of malar and maxillary bones, Hematoma complicating a procedure, Iatrogenic pulmonary embolism and infarction, Acute posthemorrhagic anemia, Pneumonia, organism unspecified, Fall from other slipping, tripping, or stumbling, Diabetes mellitus without mention of complication, type II or unspecified type, not stated as uncontrolled, Other specified surgical operations and procedures causing abnormal patient reaction, or later complication, without mention of misadventure at time of operation, Personal history of other diseases of circulatory system, Other and unspecified hyperlipidemia, Unspecified essential hypertension.

Procedures performed: Partial shoulder replacement, Transfusion of packed cells.

Medications prescribed: Hydromorphone, Sodium Chloride 0.9%  Flush, NIFEdipine CR, Atorvastatin, Furosemide, Heparin, Metoprolol, Insulin, GlipiZIDE, Pioglitazone HCl.

Abnormal lab results observed in: % Hemoglobin A1c, Glucose, Triglycerides, Bands, Lymphocytes, Neutrophils, PTT, White Blood Cells, Hematocrit, Hemoglobin.

No microbiology records available.

Average IV fluid/medication input: 534.55 units.

Average fluid output (e.g. urine): 211.6 units.

Patient transferred through units: CCU.

DRG complexity descriptions: Shoulder, Upper Arm & Forearm Procedures, MAJOR JOINT & LIMB REATTACHMENT PROCEDURES OF UPPER EXTREMITY.""",
    output_summary="""Female patient admitted on 2149-05-26 for emergency care with a primary diagnosis of a humeral fracture. Notable associated conditions included closed fractures of the humerus, facial bones, and maxillary bones; emphysema; pneumonia; iatrogenic pulmonary embolism; hematoma and acute posthemorrhagic anemia; hypertension; type II diabetes mellitus; and hyperlipidemia. The injury occurred following a fall.

The patient underwent a partial shoulder replacement and received transfusions of packed red cells. Medications included analgesics (Hydromorphone), anticoagulants (Heparin), cardiovascular agents (Metoprolol, Nifedipine, Atorvastatin), diuretics (Furosemide), and diabetes management drugs (Insulin, Glipizide, Pioglitazone).

Laboratory abnormalities were observed in glucose metabolism, lipid profile, coagulation (PTT), and complete blood count parameters including WBC, hemoglobin, and hematocrit.

No microbiology results were available. The patient stayed 2.14 days in the CCU with an average IV input of 534.55 units and output of 211.6 units.

DRG classification: Major joint and limb reattachment procedures of upper extremity."""
)

In [41]:
add_manual_entry(
    subject_id=10019,
    target_audience="standard",
    input_summary="""Patient is a M admitted on 2163-05-14 for emergency care with a primary diagnosis of 'ALCOHOLIC HEPATITIS'. Ethnicity: WHITE, Marital Status: DIVORCED, Insurance: Medicare. Religion: CATHOLIC. Length of ICU stay: 1.2938 days.

Diagnoses include: Unspecified septicemia, Acute respiratory failure, Acute pancreatitis, Other and unspecified alcohol dependence, unspecified, Blood in stool, Acute kidney failure with lesion of tubular necrosis, Hepatic encephalopathy, Other shock without mention of trauma, Severe sepsis, Acute alcoholic hepatitis, Unspecified essential hypertension, Asthma, unspecified type, unspecified.

Procedures performed: Continuous invasive mechanical ventilation for less than 96 consecutive hours, Infusion of vasopressor agent, Hemodialysis, Transfusion of packed cells.

Medications prescribed: Propofol, Heparin Flush CVL  (100 units/ml), Phytonadione, Albumin 25% (12.5gm), Dexamethasone, Calcium Gluconate, Cisatracurium Besylate, Artificial Tear Ointment, Sodium Bicarbonate, D5W.

Abnormal lab results observed in: Alanine Aminotransferase (ALT), Albumin, Alkaline Phosphatase, Ammonia, Asparate Aminotransferase (AST), Bicarbonate, Bilirubin, Total, Calcium, Total, Creatinine, Glucose.

Microbiology cultures detected organisms: GRAM POSITIVE COCCUS(COCCI), ENTEROCOCCUS FAECALIS, STREPTOCOCCUS SPECIES, ENTEROCOCCUS SP..

Average IV fluid/medication input: 52.12 units.

Average fluid output (e.g. urine): 258.55 units.

Patient transferred through units: MICU.

DRG complexity descriptions: SEPTICEMIA AGE >17.""",
    output_summary="""Male patient admitted on 2163-05-14 for emergency care with a primary diagnosis of alcoholic hepatitis. Significant comorbidities included severe sepsis, hepatic encephalopathy, acute respiratory failure, acute pancreatitis, acute kidney injury (tubular necrosis), shock (non-traumatic), asthma, hypertension, and alcohol dependence. Gastrointestinal symptoms included hematochezia (blood in stool).

The patient received mechanical ventilation (duration <96 hours), vasopressors, hemodialysis, and packed red blood cell transfusion.

Medications administered included sedatives (Propofol), anticoagulants (Heparin), corticosteroids (Dexamethasone), vitamin K (Phytonadione), calcium supplementation, cisatracurium (neuromuscular blocker), artificial tear ointment, sodium bicarbonate, and D5W for fluid therapy.

Abnormal labs were noted in liver enzymes (ALT, AST, Alkaline Phosphatase, Bilirubin), electrolytes (Bicarbonate, Calcium), renal function markers (Creatinine), and glucose levels.

Microbiology cultures revealed the presence of Gram-positive cocci, including Enterococcus faecalis, Streptococcus species, and Enterococcus spp.

ICU stay lasted approximately 1.29 days in the MICU. Average IV fluid/medication input was 52.12 units, and output was 258.55 units.

DRG classification: Septicemia in patients aged >17."""
)

In [44]:
add_manual_entry(
    subject_id=10026,
    target_audience="standard",
    input_summary="""Patient is a F admitted on 2195-05-17 for emergency care with a primary diagnosis of 'STROKE/TIA'. Ethnicity: WHITE, Marital Status: nan, Insurance: Medicare. Religion: OTHER. Length of ICU stay: 2.4056 days.

Diagnoses include: Intracerebral hemorrhage, Malignant essential hypertension, Urinary tract infection, site not specified, Hyposmolality and/or hyponatremia, Personal history of malignant neoplasm of breast, Unspecified hereditary and idiopathic peripheral neuropathy, Anemia, unspecified.

No procedures recorded.

Medications prescribed: Metoprolol, Enalaprilat, Famotidine, Levothyroxine Sodium, Lactulose, Heparin, Ciprofloxacin HCl, Pantoprazole, Lisinopril, Sodium Chloride 0.9%  Flush.

Abnormal lab results observed in: Amylase, Chloride, Glucose, Lipase, Sodium, Hematocrit, Hemoglobin, Neutrophils, RDW, Red Blood Cells.

Microbiology cultures detected organisms: ENTEROBACTERIACEAE, ESCHERICHIA COLI.

Average IV fluid/medication input: 132.35 units.

Average fluid output (e.g. urine): 39.02 units.

Patient transferred through units: SICU.

DRG complexity descriptions: Intracranial Hemorrhage, INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION.""",
    output_summary="""Female patient admitted on 2195-05-17 for emergency care with a primary diagnosis of stroke/transient ischemic attack (TIA), complicated by intracerebral hemorrhage. Notable comorbidities included malignant essential hypertension, urinary tract infection (site unspecified), anemia, hyponatremia, peripheral neuropathy (hereditary/idiopathic), and a history of malignant neoplasm of the breast.

The patient did not undergo any documented procedures during the ICU stay.

Pharmacologic management included antihypertensives (Metoprolol, Enalaprilat, Lisinopril), gastrointestinal protectants (Famotidine, Pantoprazole), anticoagulants (Heparin), antibiotics (Ciprofloxacin), thyroid hormone replacement (Levothyroxine), and other supportive agents (Lactulose, Sodium Chloride flush).

Abnormal laboratory results were observed in pancreatic enzymes (Amylase, Lipase), electrolytes (Sodium, Chloride), glucose, hematologic indices (Hemoglobin, Hematocrit, RBC count, RDW, Neutrophils).

Microbiology cultures identified Escherichia coli, classified under Enterobacteriaceae, indicating a likely urinary source of infection.

The patient stayed in the SICU for approximately 2.41 days, with average IV input of 132.35 units and output of 39.02 units.

DRG classification: Intracranial hemorrhage or cerebral infarction."""
)

In [47]:
add_manual_entry(
    subject_id=10027,
    target_audience="standard",
    input_summary="""Patient is a F admitted on 2190-07-13 for elective care with a primary diagnosis of ' MITRAL REGURGITATION;CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS GRAFT WITH MVR  ? MITRAL VALVE REPLACEMENT /SDA'. Ethnicity: WHITE, Marital Status: MARRIED, Insurance: Medicare. Religion: CATHOLIC. Length of ICU stay: 7.2616 days.

Diagnoses include: Congestive heart failure, unspecified, Mitral valve disorders, Diseases of tricuspid valve, Other heart block, Atrial fibrillation, Chronic kidney disease, unspecified, Coronary atherosclerosis of native coronary artery, Unspecified essential hypertension, Diabetes mellitus without mention of complication, type II or unspecified type, not stated as uncontrolled, Other persistent mental disorders due to conditions classified elsewhere, Rectal prolapse, Abnormal coagulation profile.

Procedures performed: Open and other replacement of mitral valve with tissue graft, (Aorto)coronary bypass of two coronary arteries, Extracorporeal circulation auxiliary to open heart surgery, Diagnostic ultrasound of heart, Initial insertion of transvenous leads [electrodes] into atrium and ventricle, Initial insertion of dual-chamber device, Other electric countershock of heart, Manual reduction of rectal prolapse, Venous catheterization, not elsewhere classified, Transfusion of packed cells, Transfusion of platelets, Transfusion of other serum.

Medications prescribed: Potassium Chloride, Docusate Sodium, Ranitidine, Aspirin EC, Warfarin, Acetaminophen, Oxycodone-Acetaminophen, Milk of Magnesia, Bisacodyl, Propofol.

Abnormal lab results observed in: Alanine Aminotransferase (ALT), Asparate Aminotransferase (AST), Creatinine, Urea Nitrogen, Hematocrit, Hemoglobin, MCH, MCV, RDW, % Hemoglobin A1c.

No microbiology records available.

Average IV fluid/medication input: 38.22 units.

Average fluid output (e.g. urine): 56.28 units.

Patient transferred through units: CSRU.

DRG complexity descriptions: Cardiac Valve Procedures w/o Cardiac Catheterization, CARDIAC VALVE & OTHER MAJOR CARDIOTHORACIC PROC WITHOUT CARDIAC CATHETER.""",
    output_summary="""Female patient admitted on 2190-07-13 for elective surgical management of mitral regurgitation and coronary artery disease. The patient underwent coronary artery bypass grafting (CABG) and mitral valve replacement (MVR) using a tissue graft.

Pre-existing conditions included congestive heart failure, mitral and tricuspid valve disorders, atrial fibrillation, heart block, chronic kidney disease, coronary atherosclerosis, hypertension, type II diabetes mellitus, and a history of rectal prolapse and coagulation abnormalities. A mental disorder due to underlying medical conditions was also documented.

The patient underwent multiple procedures, including dual-chamber pacemaker placement, open-heart surgery with extracorporeal circulation, diagnostic echocardiography, venous catheterization, and rectal prolapse reduction. Blood product transfusions (packed cells, platelets, and serum) were administered during the ICU stay.

Pharmacologic treatment included anticoagulants (Warfarin, Aspirin), analgesics (Oxycodone-Acetaminophen, Acetaminophen), sedatives (Propofol), GI agents (Docusate Sodium, Milk of Magnesia, Bisacodyl, Ranitidine), and electrolyte replacement (Potassium Chloride).

Abnormal laboratory findings involved liver enzymes (ALT, AST), renal markers (Creatinine, Urea Nitrogen), and hematologic indices (Hemoglobin, Hematocrit, MCV, MCH, RDW, Hemoglobin A1c).

The patient was managed postoperatively in the CSRU with an ICU stay of approximately 7.26 days. IV fluid/medication input was 38.22 units, and output was 56.28 units.

DRG classification: Cardiac valve and major cardiothoracic procedures without cardiac catheterization."""
)

In [49]:
add_manual_entry(
    subject_id=10029,
    target_audience="standard",
    input_summary="""Patient is a M admitted on 2139-09-22 for emergency care with a primary diagnosis of 'SYNCOPE;TELEMETRY'. Ethnicity: WHITE, Marital Status: DIVORCED, Insurance: Medicare. Religion: PROTESTANT QUAKER. Length of ICU stay: 2.2624 days.

Diagnoses include: Streptococcal septicemia, Other complications due to renal dialysis device, implant, and graft, Cholangitis, Calculus of bile duct without mention of cholecystitis, with obstruction, Atrial fibrillation, End stage renal disease, Urinary tract infection, site not specified, Septic shock, Other pulmonary insufficiency, not elsewhere classified, Thrombocytopenia, unspecified, Unspecified pleural effusion, Hypertensive chronic kidney disease, unspecified, with chronic kidney disease stage V or end stage renal disease, Congestive heart failure, unspecified, Mechanical complication due to other implant and internal device, not elsewhere classified, Cervical spondylosis with myelopathy, History of fall, Neoplasm of uncertain behavior of bone and articular cartilage, Delirium due to conditions classified elsewhere, Severe sepsis, Automatic implantable cardiac defibrillator in situ, Personal history of malignant neoplasm of prostate.

Procedures performed: Revision of arteriovenous shunt for renal dialysis, Endoscopic retrograde cholangiopancreatography [ERCP], Replacement of stent (tube) in biliary or pancreatic duct, Infusion of vasopressor agent, Hemodialysis, Contrast myelogram, Arteriography of other specified sites, Venous catheterization for renal dialysis.

Medications prescribed: NS, D5W, LR, NS (Mini Bag Plus), Isotonic Sodium Chloride, D5 1/2NS, Levofloxacin, Vancomycin HCl, Metronidazole, Piperacillin-Tazobactam Na.

Abnormal lab results observed in: Creatine Kinase (CK), Creatinine, Glucose, Troponin T, Urea Nitrogen, Hemoglobin, Lymphocytes, MCH, Neutrophils, RDW.

Microbiology cultures detected organisms: SERRATIA MARCESCENS, ENTEROCOCCUS SP., ENTEROCOCCUS FAECALIS, ESCHERICHIA COLI.

Average IV fluid/medication input: 85.66 units.

Average fluid output (e.g. urine): 18.68 units.

Patient transferred through units: SICU.

DRG complexity descriptions: Infectious & Parasitic Diseases Including HIV W O.R. Procedure, INFECTIOUS & PARASITIC DISEASES W OR PROCEDURE.""",
    output_summary="""Male patient admitted on 2139-09-22 for emergency care with a primary diagnosis of syncope, managed under telemetry.

The patient presented with a complex medical background including end-stage renal disease, atrial fibrillation, septic shock, streptococcal septicemia, cholangitis, bile duct obstruction, congestive heart failure, hypertensive kidney disease, and complications from prior renal dialysis devices. Notable comorbidities included cervical spondylosis with myelopathy, thrombocytopenia, urinary tract infection, pleural effusion, pulmonary insufficiency, and a history of malignancy (prostate), bone neoplasm, falls, and delirium.

Procedures performed included revision of an arteriovenous dialysis shunt, ERCP with stent replacement, infusion of vasopressors, hemodialysis, contrast myelogram, arteriography, and venous catheterization for dialysis.

Medications administered included IV fluids (Normal Saline, D5W, Lactated Ringer's, D5 ½ NS), and broad-spectrum antibiotics including Levofloxacin, Vancomycin, Metronidazole, and Piperacillin-Tazobactam.

Abnormal lab findings were noted in markers of muscle and renal injury (CK, Creatinine, Urea Nitrogen), glucose levels, cardiac enzyme Troponin T, and hematologic indices (Hemoglobin, RDW, Neutrophils, MCH, Lymphocytes).

Microbiology cultures revealed polymicrobial infection, including Serratia marcescens, Enterococcus faecalis, Enterococcus sp., and Escherichia coli.

The patient was managed in the SICU for approximately 2.26 days, receiving an average IV fluid/medication input of 85.66 units and output of 18.68 units.

DRG classification: Infectious and parasitic diseases (including HIV) with an operating room procedure."""
)

In [51]:
add_manual_entry(
    subject_id=10032,
    target_audience="standard",
    input_summary="""Patient is a M admitted on 2138-04-02 for emergency care with a primary diagnosis of 'RIGHT HUMEROUS FRACTURE'. Ethnicity: WHITE, Marital Status: WIDOWED, Insurance: Medicare. Religion: CATHOLIC. Length of ICU stay: 3.712 days.

Diagnoses include: Other closed fracture of lower end of humerus, Secondary malignant neoplasm of bone and bone marrow, Pneumonitis due to inhalation of food or vomitus, Diastolic heart failure, unspecified, Congestive heart failure, unspecified, Acute posthemorrhagic anemia, Thrombocytopenia, unspecified, Acute kidney failure, unspecified.

Procedures performed: Open reduction of fracture with internal fixation, humerus, Transfusion of packed cells, Insertion of two vascular stents, Transposition of cranial and peripheral nerves, Venous catheterization, not elsewhere classified.

Medications prescribed: Prochlorperazine, Naproxen, Furosemide, Docusate Sodium, Senna, Dronabinol, Tizanidine HCl, Levofloxacin, Metronidazole, Heparin.

Abnormal lab results observed in: Prostate Specific Antigen, Urea Nitrogen, Hematocrit, Hemoglobin, PT, Red Blood Cells, Alkaline Phosphatase, Asparate Aminotransferase (AST), Chloride, Lactate Dehydrogenase (LD).

No microbiology records available.

Average IV fluid/medication input: 61.25 units.

Average fluid output (e.g. urine): 72.64 units.

Patient transferred through units: MICU.

DRG complexity descriptions: LOWER EXTREMITY & HUMERUS PROCEDURES EXCEPT HIP, FOOT, FEMUR AGE>17 COMPLICATIONS, COMORBIDITIES.""",
    output_summary="""Male patient admitted on 2138-04-02 for emergency care with a primary diagnosis of right humerus fracture.

The clinical course was complicated by multiple comorbidities including congestive and diastolic heart failure, acute posthemorrhagic anemia, thrombocytopenia, acute kidney injury, pneumonitis due to aspiration, and secondary malignant neoplasm involving the bone and marrow.

Surgical and interventional procedures included open reduction and internal fixation (ORIF) of the humerus fracture, insertion of two vascular stents, nerve transposition, venous catheterization, and transfusion of packed red blood cells.

The patient was managed with medications including analgesics (Naproxen), antiemetics (Prochlorperazine), muscle relaxants (Tizanidine), antibiotics (Levofloxacin, Metronidazole), anticoagulants (Heparin), diuretics (Furosemide), stool softeners/laxatives (Docusate, Senna), and appetite stimulant (Dronabinol).

Laboratory abnormalities included derangements in renal (Urea Nitrogen), hepatic (AST, Alkaline Phosphatase, LDH), hematologic (Hemoglobin, Hematocrit, RBCs, PT), electrolyte (Chloride), and tumor marker (PSA) profiles.

The patient received care in the MICU for approximately 3.71 days, with an average IV input of 61.25 units and output of 72.64 units.

DRG classification: Lower extremity and humerus procedures (excluding hip, foot, femur) in adults over 17 with complications or comorbidities."""
)

In [56]:
add_manual_entry(
    subject_id=10033,
    target_audience="standard",
    input_summary="""Patient is a F admitted on 2132-12-05 for emergency care with a primary diagnosis of 'RENAL FAILIURE-SYNCOPE-HYPERKALEMIA'. Ethnicity: WHITE, Marital Status: MARRIED, Insurance: Medicare. Religion: CATHOLIC. Length of ICU stay: 1.0695 days.

Diagnoses include: Toxic multinodular goiter without mention of thyrotoxic crisis or storm, Other diseases of trachea and bronchus, Acute kidney failure, unspecified, Hyperpotassemia, Atrial fibrillation, Unspecified fall, Unspecified essential hypertension, Diabetes mellitus without mention of complication, type II or unspecified type, not stated as uncontrolled, Pure hypercholesterolemia, Depressive disorder, not elsewhere classified, Late effects of cerebrovascular disease, aphasia, Peptic ulcer of unspecified site, unspecified as acute or chronic, without mention of hemorrhage or perforation, without mention of obstruction.

No procedures recorded.

Medications prescribed: Sodium Chloride 0.9%  Flush, Sodium Polystyrene Sulfonate, Metoprolol, Ranitidine, Atorvastatin, Heparin, Insulin, Venlafaxine XR, Lisinopril, Furosemide.

Abnormal lab results observed in: Creatinine, Glucose, Phosphate, Potassium, Urea Nitrogen, Potassium, Whole Blood, Thyroxine (T4), Free, Triiodothyronine (T3), Hemoglobin, Red Blood Cells.

No microbiology records available.

Average IV fluid/medication input: 104.5 units.

Average fluid output (e.g. urine): 200.0 units.

Patient transferred through units: SICU.

DRG complexity descriptions: Other Endocrine Disorders, ENDOCRINE DISORDERS WITH COMPLICATIONS, COMORBIDITIES.""",
    output_summary="""Female patient admitted on 2132-12-05 for emergency care with a primary diagnosis of renal failure, syncope, and hyperkalemia.

Significant comorbidities included toxic multinodular goiter, acute kidney injury, atrial fibrillation, hyperkalemia, essential hypertension, type II diabetes mellitus, pure hypercholesterolemia, unspecified fall, depressive disorder, tracheobronchial disease, aphasia secondary to cerebrovascular disease, and a non-complicated peptic ulcer.

No interventional procedures were documented during hospitalization.

Pharmacologic management included electrolyte-correcting agents (Sodium Polystyrene Sulfonate), cardiovascular medications (Metoprolol, Lisinopril, Atorvastatin, Heparin), diuretics (Furosemide), insulin for glycemic control, and medications addressing gastrointestinal (Ranitidine) and psychiatric conditions (Venlafaxine XR).

Laboratory findings showed abnormalities in renal function (Creatinine, Urea Nitrogen, Phosphate), electrolytes (Potassium), thyroid profile (Free T4, T3), and hematologic parameters (Hemoglobin, RBC count).

The patient had an ICU stay of 1.07 days in the SICU, with an average IV input of 104.5 units and a notably high fluid output of 200.0 units.

DRG classification: Endocrine disorders with complications or comorbidities."""
)

In [57]:
print(subject_ids)

[10006, 10011, 10013, 10017, 10019, 10026, 10027, 10029, 10032, 10033, 10035, 10036, 10038, 10040, 10042, 10043, 10044, 10045, 10046, 10056, 10059, 10061, 10064, 10065, 10067, 10069, 10074, 10076, 10083, 10088, 10089, 10090, 10093, 10094, 10098, 10101, 10102, 10104, 10106, 10111, 10112, 10114, 10117, 10119, 10120, 10124, 10126, 10127, 10130, 10132, 40124, 40177, 40204, 40277, 40286, 40304, 40310, 40456, 40503, 40595, 40601, 40612, 40655, 40687, 41795, 41914, 41976, 41983, 42033, 42066, 42075, 42135, 42199, 42231, 42275, 42281, 42292, 42302, 42321, 42346, 42367, 42412, 42430, 42458, 43735, 43746, 43748, 43779, 43798, 43827, 43870, 43879, 43881, 43909, 43927, 44083, 44154, 44212, 44222, 44228]


In [96]:
patient_data = get_patient_report(10088)
text_summary = generate_text_summary_enhanced(patient_data)
print(text_summary)

Patient is a M admitted on 2107-01-04 for emergency care with a primary diagnosis of 'SEPSIS;PNEUMONIA;TELEMETRY'. Ethnicity: WHITE, Marital Status: UNKNOWN (DEFAULT), Insurance: Medicare. Religion: UNOBTAINABLE. Length of ICU stay: 3.6885 days.

Diagnoses include: Unspecified septicemia, Subendocardial infarction, initial episode of care, Pneumonitis due to inhalation of food or vomitus, Severe sepsis, Sepsis, Septic shock, Other pulmonary insufficiency, not elsewhere classified, Urinary tract infection, site not specified, Congestive heart failure, unspecified, Acute kidney failure, unspecified, Acute respiratory failure, Acidosis, Infection with microorganisms resistant to penicillins, Paralysis agitans, Hyperosmolality and/or hypernatremia, Blood in stool, Other malignant lymphomas, unspecified site, extranodal and solid organ sites, Vascular dementia, with delirium, Bronchitis, not specified as acute or chronic, Cerebral atherosclerosis, Gastrostomy status, Aortocoronary bypass st

In [97]:
add_manual_entry(
    subject_id=10088,
    target_audience="standard",
    input_summary="""Patient is a M admitted on 2107-01-04 for emergency care with a primary diagnosis of 'SEPSIS;PNEUMONIA;TELEMETRY'. Ethnicity: WHITE, Marital Status: UNKNOWN (DEFAULT), Insurance: Medicare. Religion: UNOBTAINABLE. Length of ICU stay: 3.6885 days.

Diagnoses include: Unspecified septicemia, Subendocardial infarction, initial episode of care, Pneumonitis due to inhalation of food or vomitus, Severe sepsis, Sepsis, Septic shock, Other pulmonary insufficiency, not elsewhere classified, Urinary tract infection, site not specified, Congestive heart failure, unspecified, Acute kidney failure, unspecified, Acute respiratory failure, Acidosis, Infection with microorganisms resistant to penicillins, Paralysis agitans, Hyperosmolality and/or hypernatremia, Blood in stool, Other malignant lymphomas, unspecified site, extranodal and solid organ sites, Vascular dementia, with delirium, Bronchitis, not specified as acute or chronic, Cerebral atherosclerosis, Gastrostomy status, Aortocoronary bypass status, Methicillin susceptible Staphylococcus aureus in conditions classified elsewhere and of unspecified site, Coronary atherosclerosis of unspecified type of vessel, native or graft, Percutaneous transluminal coronary angioplasty status, Anemia of other chronic disease, Other specified cardiac dysrhythmias, Unspecified essential hypertension, Depressive disorder, not elsewhere classified, Peripheral vascular disease, unspecified, Vascular dementia, uncomplicated, Personal history of tuberculosis, Personal history of other lymphatic and hematopoietic neoplasms, Anemia, unspecified, Atherosclerosis of autologous vein bypass graft of the extremities, Diverticulosis of colon (without mention of hemorrhage), Other and unspecified hyperlipidemia.

Procedures performed: Continuous invasive mechanical ventilation for less than 96 consecutive hours, Enteral infusion of concentrated nutritional substances, Transfusion of packed cells, Venous catheterization, not elsewhere classified, Arterial catheterization.

Medications prescribed: Heparin Flush CVL  (100 units/ml), Donepezil, Atorvastatin, Artificial Tears, Docusate Sodium (Liquid), Carbidopa-Levodopa (25-250), Acetaminophen, Albuterol, Ipratropium Bromide MDI, Senna.

Abnormal lab results observed in: Amylase, Calcium, Total, Chloride, Creatinine, Glucose, Lipase, Phosphate, Sodium, Troponin T, Urea Nitrogen.

Microbiology cultures detected organisms: YEAST, STAPH AUREUS COAG +.

Average IV fluid/medication input: 118.32 units.

Average fluid output (e.g. urine): 109.46 units.

Patient transferred through units: MICU, CCU.

DRG complexity descriptions: CIRCULATORY DISORDERS WITH ACUTE MYOCARDIAL INFARCTION & MAJOR COMPLICATION, DISCHARGED ALIVE, SEPTICEMIA AGE >17.""",
    output_summary="""Male patient admitted on 2107-01-04 for emergency care due to sepsis, pneumonia, and telemetry monitoring. ICU stay lasted approximately 3.69 days with transfers between MICU and CCU. DRG classification includes “Circulatory Disorders with Acute Myocardial Infarction & Major Complication” and “Septicemia Age >17.”

The patient had a complex clinical presentation involving septic shock, severe sepsis, and pneumonitis due to aspiration. Additional conditions included subendocardial infarction, acute respiratory and kidney failure, congestive heart failure, bronchitis, hypernatremia, acidosis, and various infections including penicillin-resistant organisms and Staphylococcus aureus.

Chronic and comorbid diagnoses encompassed vascular dementia (with and without delirium), Parkinson’s disease (paralysis agitans), atherosclerosis (cerebral and peripheral), coronary artery disease, anemia of chronic disease, diverticulosis, hyperlipidemia, depressive disorder, hypertension, prior tuberculosis, and prior lymphoma.

Procedures performed included invasive ventilation (<96 hrs), enteral nutrition infusion, transfusion of packed RBCs, and arterial/venous catheterizations.

Medications administered included antiplatelets and vasodilators (carbidopa-levodopa), statins (atorvastatin), bronchodilators (albuterol, ipratropium), anticoagulants (heparin), cognitive enhancers (donepezil), and GI medications (senna, docusate, acetaminophen).

Abnormal lab findings were observed in amylase, calcium, chloride, creatinine, glucose, lipase, phosphate, sodium, troponin T, and urea nitrogen. Microbiology cultures identified yeast and Staphylococcus aureus.

IV fluid/medication input averaged 118.32 units, with an output of 109.46 units."""
)

In [95]:
add_manual_entry(
    subject_id=10083,
    target_audience="standard",
    input_summary="""Patient is a F admitted on 2192-11-20 for emergency care with a primary diagnosis of 'HYPOTENSION'. Ethnicity: WHITE, Marital Status: MARRIED, Insurance: Medicare. Religion: CATHOLIC. Length of ICU stay: 4.5761 days.

Diagnoses include: Other amyloidosis, Infection and inflammatory reaction due to other vascular device, implant, and graft, Nutritional marasmus, Urinary tract infection, site not specified, Congestive heart failure, unspecified, Nephrotic syndrome in diseases classified elsewhere, Hypovolemia, Hematemesis, Acidosis, Hyposmolality and/or hyponatremia, Bacteremia, Infection with microorganisms without mention of resistance to multiple drugs, Unspecified disorder of plasma protein metabolism, Other iatrogenic hypotension, Depressive disorder, not elsewhere classified, Other disorders of plasma protein metabolism, Streptococcus infection in conditions classified elsewhere and of unspecified site, streptococcus, group D [Enterococcus], Other esophagitis, Anemia of other chronic disease, Esophageal reflux, Bacteroides fragilis.

Procedures performed: Esophagogastroduodenoscopy [EGD] with closed biopsy, Parenteral infusion of concentrated nutritional substances, Venous catheterization, not elsewhere classified, Transfusion of packed cells.

Medications prescribed: Heparin Flush CVL  (100 units/ml), Albuterol 0.083% Neb Soln, Fluconazole, Montelukast Sodium, Pantoprazole, Multivitamins, Enoxaparin Sodium, Prednisone, Senna, Docusate Sodium.

Abnormal lab results observed in: Calcium, Total, Urea Nitrogen, Hematocrit, Hemoglobin, MCH, RDW, Red Blood Cells, Sodium, White Blood Cells, Glucose.

Microbiology cultures detected organisms: BACTEROIDES FRAGILIS GROUP, STAPHYLOCOCCUS, COAGULASE NEGATIVE, ANAEROBIC GRAM POSITIVE ROD(S), ENTEROCOCCUS FAECIUM, ENTEROCOCCUS SP..

Average IV fluid/medication input: 68.42 units.

Average fluid output (e.g. urine): 378.46 units.

Patient transferred through units: MICU, CCU.

DRG complexity descriptions: Connective Tissue Disorders, CONNECTIVE TISSUE DISORDERS WITH COMPLICATIONS, COMORBIDITIES.""",
    output_summary="""Female patient admitted on 2192-11-20 for emergency care due to hypotension. ICU stay lasted approximately 4.58 days, with transfers between MICU and CCU. DRG classification was "Connective Tissue Disorders with Complications or Comorbidities."

The patient had multiple comorbid conditions, including amyloidosis, nephrotic syndrome, and congestive heart failure. She also presented with nutritional marasmus, urinary tract infection, hyponatremia, bacteremia, hypovolemia, hematemesis, acidosis, and infections related to vascular devices and implants. Other notable conditions included depressive disorder, anemia of chronic disease, plasma protein metabolism disorders, and esophageal reflux.

Microbiology cultures revealed presence of Bacteroides fragilis, Staphylococcus coagulase-negative, Enterococcus faecium, Enterococcus species, and anaerobic gram-positive rods.

Procedures included esophagogastroduodenoscopy (EGD) with biopsy, parenteral nutritional infusion, venous catheterization, and transfusion of packed red blood cells.

Medications administered included anticoagulants (heparin, enoxaparin), antifungals (fluconazole), steroids (prednisone), GI agents (pantoprazole, senna, docusate), respiratory medications (albuterol, montelukast), and multivitamins.

Abnormal laboratory findings included derangements in calcium, sodium, glucose, urea nitrogen, hemoglobin, hematocrit, red and white blood cell counts, MCH, and RDW.

The patient received an average of 68.42 units of IV fluid/medication and had an output of 378.46 units."""
)

In [93]:
add_manual_entry(
    subject_id=10076,
    target_audience="standard",
    input_summary="""Patient is a M admitted on 2107-03-21 for emergency care with a primary diagnosis of 'LUNG CANCER;SHORTNESS OF BREATH'. Ethnicity: ASIAN, Marital Status: MARRIED, Insurance: Medicare. Religion: NOT SPECIFIED. Length of ICU stay: 7.1173 days.

Diagnoses include: Pneumonia, organism unspecified, Secondary malignant neoplasm of bone and bone marrow, Secondary malignant neoplasm of pleura, Acute kidney failure, unspecified, Malignant neoplasm of lower lobe, bronchus or lung, Secondary malignant neoplasm of lung, Acidosis, Unspecified septicemia, Severe sepsis, Acute respiratory failure, Unspecified protein-calorie malnutrition, Hyposmolality and/or hyponatremia, Anemia in neoplastic disease, Diarrhea, Antineoplastic and immunosuppressive drugs causing adverse effects in therapeutic use.

Procedures performed: Continuous invasive mechanical ventilation for 96 consecutive hours or more, Insertion of endotracheal tube, Parenteral infusion of concentrated nutritional substances, Venous catheterization, not elsewhere classified, Transfusion of packed cells, Pulmonary artery wedge monitoring.

Medications prescribed: Iso-Osmotic Dextrose, D5NS, D5W, NS, Soln., SW, Ceftriaxone, Vancomycin HCl, Piperacillin-Tazobactam Na, Azithromycin .

Abnormal lab results observed in: Granulocyte Count, Hematocrit, Hemoglobin, MCHC, Red Blood Cells, White Blood Cells, Glucose, Urea Nitrogen, Atypical Lymphocytes, Metamyelocytes.

Microbiology cultures detected organisms: ASPERGILLUS SP. NOT FUMIGATUS, FLAVUS OR NIGER.

Average IV fluid/medication input: 60.76 units.

Average fluid output (e.g. urine): 42.83 units.

Patient transferred through units: MICU.

DRG complexity descriptions: RESPIRATORY SYSTEM DIAGNOSIS WITH VENTILATOR SUPPORT.""",
    output_summary="""Male patient admitted on 2107-03-21 for emergency care with primary complaints of lung cancer and shortness of breath. ICU stay lasted approximately 7.12 days in the Medical ICU (MICU). DRG classification was "Respiratory System Diagnosis with Ventilator Support."

The patient was diagnosed with advanced lung cancer involving the lower lobe and metastasis to bone, bone marrow, pleura, and contralateral lung. Additional conditions included pneumonia (organism unspecified), acute kidney failure, acidosis, unspecified septicemia progressing to severe sepsis, and acute respiratory failure. Nutritional and electrolyte imbalances were also present, including protein-calorie malnutrition and hyponatremia. The patient also had anemia related to neoplastic disease, diarrhea, and adverse effects due to chemotherapy.

Procedures performed included continuous invasive mechanical ventilation (≥96 hours), endotracheal intubation, parenteral nutritional support, venous catheterization, pulmonary artery wedge pressure monitoring, and transfusion of packed cells.

Medications administered included multiple IV fluid formulations (Iso-Osmotic Dextrose, D5NS, D5W, NS, SW) and broad-spectrum antibiotics (ceftriaxone, vancomycin, piperacillin-tazobactam, azithromycin).

Abnormal lab findings included derangements in hematocrit, hemoglobin, MCHC, RBC, WBC, glucose, BUN, granulocytes, and presence of atypical lymphocytes and metamyelocytes.

Microbiological cultures were positive for Aspergillus species (excluding fumigatus, flavus, or niger).

The patient received an average of 60.76 units of IV fluids/medications, with a fluid output of 42.83 units."""
)

In [91]:
add_manual_entry(
    subject_id=10074,
    target_audience="standard",
    input_summary="""Patient is a M admitted on 2167-02-11 for emergency care with a primary diagnosis of 'GASTROINTESTINAL BLEED'. Ethnicity: WHITE, Marital Status: MARRIED, Insurance: Medicare. Religion: OTHER. Length of ICU stay: 0.6549 days.

Diagnoses include: Angiodysplasia of intestine with hemorrhage, Atrial fibrillation, Pneumonia, organism unspecified, Rheumatic heart failure (congestive), Mitral valve insufficiency and aortic valve stenosis, Diseases of tricuspid valve, Acute kidney failure, unspecified, Acute posthemorrhagic anemia, Chronic airway obstruction, not elsewhere classified, Other specified cardiac dysrhythmias, Long-term (current) use of anticoagulants.

Procedures performed: Initial insertion of transvenous lead [electrode] into ventricle, Initial insertion of single-chamber device, not specified as rate responsive, Injection or infusion of other therapeutic or prophylactic substance, Arteriography of other intra-abdominal arteries, Transfusion of packed cells.

Medications prescribed: Sodium Chloride 0.9%  Flush, Acetaminophen, Albuterol, Fluticasone Propionate 110mcg, Metoprolol, Digoxin, Magnesium Sulfate, Albuterol 0.083% Neb Soln, Ipratropium Bromide Neb, Pantoprazole.

Abnormal lab results observed in: Chloride, Creatinine, Glucose, Urea Nitrogen, Hematocrit, Hemoglobin, Lymphocytes, MCH, Neutrophils, PT.

No microbiology records available.

Average IV fluid/medication input: 232.0 units.

Average fluid output (e.g. urine): 105.83 units.

Patient transferred through units: MICU.

DRG complexity descriptions: EXTENSIVE OPERATING ROOM PROCEDURE UNRELATED TO PRINCIPAL DIAGNOSIS.""",
    output_summary="""Male patient admitted on 2167-02-11 for emergency care due to gastrointestinal bleeding. ICU stay lasted approximately 0.65 days in the Medical ICU (MICU). The DRG category was "Extensive Operating Room Procedure Unrelated to Principal Diagnosis."

Diagnoses included intestinal angiodysplasia with hemorrhage, atrial fibrillation, pneumonia (organism unspecified), rheumatic congestive heart failure, mitral insufficiency, aortic stenosis, tricuspid valve disease, acute kidney failure, posthemorrhagic anemia, chronic airway obstruction, and other specified cardiac arrhythmias. Patient also had a history of long-term anticoagulant use.

Procedures performed included insertion of a single-chamber cardiac device with ventricular lead, intra-abdominal arteriography, infusion of therapeutic substances, and transfusion of packed cells.

Medications administered included sodium chloride flush, acetaminophen, beta-blockers (metoprolol), digoxin, magnesium sulfate, bronchodilators (albuterol, ipratropium), inhaled corticosteroids (fluticasone), and pantoprazole.

Laboratory abnormalities were observed in chloride, creatinine, glucose, blood urea nitrogen (BUN), hematocrit, hemoglobin, PT, MCH, neutrophils, and lymphocytes. No microbiological findings were recorded.

The patient received an average of 232.0 units of IV fluid/medication, with a fluid output of 105.83 units."""
)

In [89]:
add_manual_entry(
    subject_id=10069,
    target_audience="standard",
    input_summary="""Patient is a F admitted on 2188-02-08 for emergency care with a primary diagnosis of 'SEIZURE'. Ethnicity: UNKNOWN/NOT SPECIFIED, Marital Status: nan, Insurance: Medicaid. Religion: nan. Length of ICU stay: 15.041 days.

Diagnoses include: Epilepsy, unspecified, with intractable epilepsy, Acute respiratory failure, Coma, Acidosis, Pneumonitis due to inhalation of food or vomitus, Unspecified intellectual disabilities, Congestive heart failure, unspecified, Diabetes mellitus without mention of complication, type II or unspecified type, not stated as uncontrolled.

Procedures performed: Parenteral infusion of concentrated nutritional substances, Enteral infusion of concentrated nutritional substances, Fiber-optic bronchoscopy, Continuous invasive mechanical ventilation for 96 consecutive hours or more, Insertion of endotracheal tube, Venous catheterization, not elsewhere classified, Spinal tap, Incision of lung.

No prescriptions data available.

Abnormal lab results observed in: Glucose, Lymphocytes, MCH, Neutrophils, Red Blood Cells, RBC, Urobilinogen, Chloride, Phosphate, Hematocrit.

Microbiology cultures detected organisms: STAPH AUREUS COAG +.

Average IV fluid/medication input: 134.44 units.

Average fluid output (e.g. urine): 189.67 units.

Patient transferred through units: CCU.

DRG complexity descriptions: SEIZURE & HEADACHE AGE >17 WITH COMPLICATIONS, COMORBIDITIES.""",
    output_summary="""Female patient admitted on 2188-02-08 for emergency care with a primary diagnosis of seizure. ICU stay lasted approximately 15.04 days in the Cardiac Care Unit (CCU). The DRG category was "Seizure & Headache Age >17 with Complications/Comorbidities."

Diagnoses included intractable epilepsy, acute respiratory failure, coma, acidosis, aspiration pneumonitis, unspecified intellectual disabilities, congestive heart failure, and type II diabetes mellitus without complication.

Procedures performed included bronchoscopy, endotracheal intubation, spinal tap, lung incision, invasive mechanical ventilation (>96 hours), and both parenteral and enteral nutritional support. Venous catheterization was also performed.

Laboratory abnormalities were noted in glucose, phosphate, chloride, red blood cells (RBC), hematocrit, MCH, lymphocyte/neutrophil counts, and urobilinogen. Microbiology cultures detected Staphylococcus aureus (coagulase-positive).

The patient received an average of 134.44 units of IV fluid/medications and had a fluid output of 189.67 units."""
)

In [87]:
add_manual_entry(
    subject_id=10067,
    target_audience="standard",
    input_summary="""Patient is a M admitted on 2130-10-06 for emergency care with a primary diagnosis of 'S/P MOTORCYCLE ACCIDENT'. Ethnicity: OTHER, Marital Status: nan, Insurance: Private. Religion: UNOBTAINABLE. Length of ICU stay: 0.1059 days.

Diagnoses include: Injury to liver without mention of open wound into cavity, laceration, major, Concussion with prolonged loss of consciousness, without return to pre-existing conscious level, Traumatic shock, Cardiac arrest, Acute posthemorrhagic anemia, Other motor vehicle nontraffic accident involving collision with stationary object injuring motorcyclist, Injury to other gastrointestinal sites, without mention of open wound into cavity, Other acquired deformity of other parts of limb.

Procedures performed: Closure of laceration of liver, Pericardiotomy, Insertion of intercostal catheter for drainage, Cardiopulmonary resuscitation, not otherwise specified, Insertion of endotracheal tube, Continuous invasive mechanical ventilation for less than 96 consecutive hours, Transfusion of packed cells.

No prescriptions data available.

Abnormal lab results observed in: Anion Gap, Bicarbonate, Chloride, Sodium, Fibrinogen, Functional, Hematocrit, Hemoglobin, Platelet Count, PT, PTT.

No microbiology records available.

No input event records available.

No output event records available.

Patient transferred through units: TSICU.

DRG complexity descriptions: OTHER OPERATING ROOM PROCEDURES FOR MULTIPLE SIGNIFICANT TRAUMA.""",
    output_summary="""Male patient admitted on 2130-10-06 following a motorcycle accident, presenting with major traumatic injuries. ICU stay in the Trauma Surgical ICU (TSICU) lasted approximately 0.11 days. The DRG category was operating room procedures for multiple significant trauma.

Diagnoses included major liver laceration without cavity penetration, concussion with prolonged unconsciousness, traumatic shock, cardiac arrest, acute posthemorrhagic anemia, and injury to gastrointestinal organs. The accident was classified as a nontraffic motor vehicle collision involving a stationary object.

Surgical and emergency procedures included closure of liver laceration, pericardiotomy, insertion of intercostal drainage catheter, endotracheal intubation, cardiopulmonary resuscitation (CPR), and mechanical ventilation for under 96 hours. Packed red blood cell transfusion was administered.

Abnormal lab results indicated disturbances in electrolytes (sodium, chloride, bicarbonate), coagulation parameters (PT, PTT, fibrinogen), and hematologic values (hemoglobin, hematocrit, platelets, anion gap).

No medication, microbiology, fluid input/output, or prescription records were available."""
)

In [85]:
add_manual_entry(
    subject_id=10065,
    target_audience="standard",
    input_summary="""Patient is a F admitted on 2189-09-08 for elective care with a primary diagnosis of 'ESOPHAGEAL CANCER/SDA'. Ethnicity: WHITE, Marital Status: SINGLE, Insurance: Private. Religion: CATHOLIC. Length of ICU stay: 2.2559 days.

Diagnoses include: Malignant neoplasm of cardia, Secondary malignant neoplasm of other digestive organs and spleen, Other vascular complications of medical care, not elsewhere classified, Chronic kidney disease, unspecified, Secondary and unspecified malignant neoplasm of intra-abdominal lymph nodes, Rheumatoid arthritis, Esophageal reflux, Pure hypercholesterolemia, Unspecified essential hypertension, Gout, unspecified, Osteoporosis, unspecified.

Procedures performed: Other total gastrectomy, Other enterostomy, Enteral infusion of concentrated nutritional substances.

Medications prescribed: Heparin Flush CVL  (100 units/ml), Heparin, Metoprolol, Morphine Sulfate, Insulin, Neostigmine, Glycopyrrolate, Magnesium Sulfate, Calcium Gluconate, Acetaminophen (Liquid).

Abnormal lab results observed in: Hematocrit, Hemoglobin, Platelet Count, RDW, Red Blood Cells, pCO2, pO2, Free Calcium, Glucose, Potassium, Whole Blood.

No microbiology records available.

Average IV fluid/medication input: 85.79 units.

Average fluid output (e.g. urine): 58.29 units.

Patient transferred through units: CSRU.

DRG complexity descriptions: Major Stomach, Esophageal & Duodenal Procedures, STOMACH, ESOPHAGEAL & DUODENAL PROC AGE >17 W CC W/O MAJOR GI DX.""",
    output_summary="""Female patient admitted on 2189-09-08 for elective surgical management of esophageal cancer. ICU stay lasted approximately 2.26 days in the Cardiac Surgery Recovery Unit (CSRU). The DRG category was major stomach, esophageal, and duodenal procedures with complications.

Diagnoses included esophageal cancer (malignant neoplasm of the cardia), with metastases to intra-abdominal lymph nodes and other digestive organs. Coexisting conditions included chronic kidney disease, rheumatoid arthritis, hypertension, esophageal reflux, osteoporosis, gout, and hypercholesterolemia.

Surgical procedures performed included total gastrectomy, creation of an enterostomy, and initiation of enteral nutritional support.

Medications administered included anticoagulants (heparin, heparin flush), cardiovascular agents (metoprolol, magnesium sulfate, calcium gluconate), insulin, pain management (morphine sulfate, acetaminophen), and gastrointestinal motility agents (neostigmine, glycopyrrolate).

Abnormal lab results were observed in hematologic parameters (hematocrit, hemoglobin, RBC, RDW, platelets), electrolytes (potassium, calcium), blood gases (pCO₂, pO₂), and glucose.

No microbiological cultures were recorded. Average IV fluid/medication input was 85.79 units with an output of 58.29 units."""
)

In [83]:
add_manual_entry(
    subject_id=10064,
    target_audience="standard",
    input_summary="""Patient is a M admitted on 2127-03-19 for emergency care with a primary diagnosis of 'SUBDURAL HEMATOMA/S/P FALL'. Ethnicity: WHITE, Marital Status: nan, Insurance: Medicare. Religion: UNOBTAINABLE. Length of ICU stay: 0.207 days.

Diagnoses include: Closed fracture of base of skull with subarachnoid, subdural, and extradural hemorrhage, with prolonged [more than 24 hours] loss of consciousness, without return to pre-existing conscious level, Traumatic hemothorax without mention of open wound into thorax, Atrial fibrillation, Other fall from one level to another in water transport injuring other specified person.

No procedures recorded.

Medications prescribed: NS, D5W, LR, Phenylephrine, Insulin Human Regular, Esmolol, Sodium Chloride 0.9%  Flush, Insulin, Epinephrine HCl.

Abnormal lab results observed in: Amylase, Urea Nitrogen, Fibrinogen, Functional, Hematocrit, Hemoglobin, INR(PT), MCH, PT, Red Blood Cells, White Blood Cells.

No microbiology records available.

Average IV fluid/medication input: 293.75 units.

Average fluid output (e.g. urine): 426.67 units.

Patient transferred through units: TSICU.

DRG complexity descriptions: OTHER MULTIPLE SIGNIFICANT TRAUMA, Multiple Significant Trauma W/O O.R. Procedure.""",
    output_summary="""Male patient admitted on 2127-03-19 for emergency care following a fall resulting in a subdural hematoma. The ICU stay lasted approximately 5 hours, with care provided in the Trauma Surgical ICU (TSICU). The case was classified under DRG as multiple significant trauma without operating room procedures.

Primary diagnoses included a skull base fracture associated with subarachnoid, subdural, and extradural hemorrhages, accompanied by prolonged unconsciousness without return to baseline. Additional diagnoses included traumatic hemothorax, atrial fibrillation, and injury from a fall involving water transport.

No surgical procedures were recorded during the admission.

Medications administered included crystalloid fluids (NS, D5W, LR), vasopressors and cardiac medications (phenylephrine, esmolol, epinephrine), insulin, and sodium chloride flushes.

Abnormal lab findings were noted in blood chemistry and hematologic markers, including amylase, urea nitrogen, coagulation markers (PT, INR, fibrinogen), red and white blood cell counts, hemoglobin, hematocrit, and MCH.

No microbiology data were available. The patient received 293.75 units of IV fluids or medications and had a fluid output of 426.67 units."""
)

In [81]:
add_manual_entry(
    subject_id=10061,
    target_audience="standard",
    input_summary="""Patient is a F admitted on 2107-01-16 for emergency care with a primary diagnosis of 'CHEST PAIN/ CATH'. Ethnicity: WHITE, Marital Status: WIDOWED, Insurance: Medicare. Religion: CATHOLIC. Length of ICU stay: 24.9968 days.

Diagnoses include: Congestive heart failure, unspecified, Subendocardial infarction, initial episode of care, Unspecified septicemia, Cardiogenic shock, Other shock without mention of trauma, Acute respiratory failure, Acute kidney failure, unspecified, Acute and subacute necrosis of liver, Pneumonia due to escherichia coli [E. coli].

Procedures performed: Combined right and left heart cardiac catheterization, Coronary arteriography using two catheters, Continuous invasive mechanical ventilation for 96 consecutive hours or more, Closed [endoscopic] biopsy of large intestine, Esophagogastroduodenoscopy [EGD] with closed biopsy, Venous catheterization, not elsewhere classified, Enteral infusion of concentrated nutritional substances, Parenteral infusion of concentrated nutritional substances.

Medications prescribed: Senna, Levofloxacin, Metronidazole, Heparin, Insulin, Potassium Chloride, Furosemide, Calcium Gluconate, Amiodarone HCl, Phytonadione.

Abnormal lab results observed in: Calculated Total CO2, pH, pO2, Albumin, Anion Gap, Bicarbonate, Calcium, Total, Creatine Kinase (CK), Creatine Kinase, MB Isoenzyme, Creatinine.

Microbiology cultures detected organisms: ESCHERICHIA COLI, YEAST, GRAM NEGATIVE ROD(S).

Average IV fluid/medication input: 48.59 units.

Average fluid output (e.g. urine): 159.35 units.

Patient transferred through units: CCU.

DRG complexity descriptions: CIRCULATORY DISORDERS WITH ACUTE MYOCARDIAL INFARCTION & MAJOR COMPLICATION, DISCHARGED ALIVE.""",
    output_summary="""Female patient admitted on 2107-01-16 for emergency evaluation of chest pain and cardiac catheterization. She remained in the Coronary Care Unit (CCU) for approximately 25 days. The DRG classification corresponds to circulatory disorders with acute myocardial infarction and major complications, with the patient discharged alive.

Primary diagnoses included subendocardial myocardial infarction (initial episode), congestive heart failure, cardiogenic shock, unspecified septicemia, E. coli pneumonia, acute respiratory failure, acute kidney injury, and hepatic necrosis. She also experienced other forms of non-traumatic shock.

The patient underwent right and left heart catheterization, coronary arteriography (dual catheter), prolonged mechanical ventilation (>96 hours), enteral and parenteral nutritional infusions, venous catheterization, EGD with biopsy, and colonoscopic biopsy.

Medications included cardiovascular and antimicrobial agents such as amiodarone, levofloxacin, metronidazole, heparin, insulin, potassium chloride, furosemide, calcium gluconate, phytonadione, and senna.

Lab abnormalities were present in acid-base balance (CO₂, pH), oxygenation (pO₂), metabolic and renal markers (albumin, anion gap, creatinine, CK-MB), and electrolytes (calcium, bicarbonate).

Microbiology cultures were positive for Escherichia coli, yeast, and gram-negative rods. The patient received an average of 48.59 units of IV fluids/medications with a fluid output of 159.35 units."""
)

In [79]:
add_manual_entry(
    subject_id=10059,
    target_audience="standard",
    input_summary="""Patient is a M admitted on 2150-08-07 for emergency care with a primary diagnosis of 'VARICEAL BLEED'. Ethnicity: WHITE, Marital Status: MARRIED, Insurance: Medicare. Religion: CATHOLIC. Length of ICU stay: 1.7806 days.

Diagnoses include: Cirrhosis of liver without mention of alcohol, Acute kidney failure, unspecified, Portal vein thrombosis, Unspecified viral hepatitis C without hepatic coma, Iron deficiency anemia secondary to blood loss (chronic), Esophageal varices in diseases classified elsewhere, with bleeding, Thrombocytopenia, unspecified, Unspecified septicemia, Severe sepsis, Portal hypertension, Septic shock, Unspecified essential hypertension, Acute respiratory failure, Hepatorenal syndrome, Chronic kidney disease, unspecified.

Procedures performed: Endoscopic excision or destruction of lesion or tissue of esophagus, Other endoscopy of small intestine, Transfusion of packed cells, Arterial catheterization, Venous catheterization, not elsewhere classified, Continuous invasive mechanical ventilation for 96 consecutive hours or more, Parenteral infusion of concentrated nutritional substances, Venous catheterization for renal dialysis, Hemodialysis, Percutaneous abdominal drainage.

Medications prescribed: Sodium Chloride 0.9%  Flush, Dolasetron Mesylate, Prochlorperazine, Insulin, Sodium Polystyrene Sulfonate, Lidocaine Jelly 2% (Urojet), Pantoprazole, Ciprofloxacin HCl, Nadolol, Spironolactone.

Abnormal lab results observed in: Bicarbonate, Chloride, Creatinine, Glucose, Phosphate, Potassium, Sodium, Urea Nitrogen, Hematocrit, Hemoglobin.

No microbiology records available.

Average IV fluid/medication input: 27.76 units.

Average fluid output (e.g. urine): 97.28 units.

Patient transferred through units: MICU.

DRG complexity descriptions: CIRRHOSIS & ALCOHOLIC HEPATITIS, Other Disorders Of The Liver.""",
    output_summary="""Male patient admitted on 2150-08-07 for emergency care with a primary diagnosis of variceal bleeding. He was managed in the Medical ICU (MICU) for approximately 1.78 days. DRG classification includes cirrhosis and other disorders of the liver.

Clinical diagnoses included cirrhosis of the liver (non-alcoholic), portal vein thrombosis, portal hypertension, esophageal varices with active bleeding, hepatitis C infection (without hepatic coma), and complications such as acute kidney injury, chronic kidney disease, hepatorenal syndrome, iron-deficiency anemia due to chronic blood loss, thrombocytopenia, septicemia, severe sepsis, septic shock, and acute respiratory failure.

The patient underwent endoscopic treatment for esophageal bleeding, endoscopy of the small intestine, dialysis-related catheterizations, packed cell transfusions, mechanical ventilation (for >96 hours), hemodialysis, arterial line placement, and parenteral nutrition.

Medications administered included pantoprazole, ciprofloxacin, insulin, nadolol, spironolactone, dolasetron, and sodium polystyrene sulfonate. Abnormal laboratory results were noted in renal markers (creatinine, urea nitrogen), electrolytes (sodium, potassium, chloride, phosphate, bicarbonate), glucose, and hematologic parameters (hematocrit, hemoglobin).

There were no documented microbiology results. The average IV input was 27.76 units, with a fluid output of 97.28 units."""
)

In [77]:
add_manual_entry(
    subject_id=10056,
    target_audience="standard",
    input_summary="""Patient is a F admitted on 2129-05-02 for emergency care with a primary diagnosis of 'SEPSIS'. Ethnicity: WHITE, Marital Status: UNKNOWN (DEFAULT), Insurance: Medicare. Religion: CHRISTIAN SCIENTIST. Length of ICU stay: 1.0491 days.

Diagnoses include: Unspecified septicemia, Urinary tract infection, site not specified, Acute kidney failure with lesion of tubular necrosis, Septic shock, Acute diastolic heart failure, Congestive heart failure, unspecified, Hyposmolality and/or hyponatremia, Severe sepsis.

Procedures performed: Transfusion of packed cells.

Medications prescribed: Acetaminophen, Pantoprazole, Docusate Sodium, Senna, Atorvastatin, Insulin, Dextrose 50%, Heparin Flush CVL  (100 units/ml), Fludrocortisone Acetate, Heparin.

Abnormal lab results observed in: Albumin, Amylase, Bilirubin, Direct, Bilirubin, Total, Calcium, Total, Chloride, Creatine Kinase (CK), Creatinine, Glucose, Lactate Dehydrogenase (LD).

No microbiology records available.

Average IV fluid/medication input: 155.41 units.

Average fluid output (e.g. urine): 146.6 units.

Patient transferred through units: MICU.

DRG complexity descriptions: SEPTICEMIA AGE >17.""",
    output_summary="""Female patient admitted on 2129-05-02 for emergency care with a primary diagnosis of sepsis. She was managed in the Medical ICU (MICU) with a total ICU stay of approximately 1.05 days. DRG classification aligns with septicemia in patients over 17 years of age.

The clinical course included diagnoses of septicemia of unspecified origin, septic shock, severe sepsis, acute kidney injury with tubular necrosis, acute diastolic and congestive heart failure, urinary tract infection, and electrolyte imbalance (hyposmolality/hyponatremia). A blood transfusion was administered.

Medications prescribed included acetaminophen, insulin, atorvastatin, pantoprazole, fludrocortisone, heparin, and dextrose. Abnormal lab values were noted in renal function markers (creatinine, LDH), liver function (bilirubin), electrolytes (chloride, calcium), glucose levels, and serum proteins (albumin, amylase).

No microbiology cultures were documented. The patient received an average of 155.41 units of IV input, with a corresponding output of 146.6 units."""
)

In [75]:
add_manual_entry(
    subject_id=10046,
    target_audience="standard",
    input_summary="""Patient is a F admitted on 2194-07-26 for emergency care with a primary diagnosis of 'BRAIN METASTASES'. Ethnicity: WHITE, Marital Status: SINGLE, Insurance: Medicaid. Religion: CATHOLIC. Length of ICU stay: 2.8833 days.

Diagnoses include: Secondary malignant neoplasm of brain and spinal cord, Other convulsions, Personal history of malignant neoplasm of other parts of uterus.

Procedures performed: Other operations on extraocular muscles and tendons, Other excision or destruction of lesion or tissue of brain.

No prescriptions data available.

Abnormal lab results observed in: Albumin, Chloride, Globulin, Glucose, Hematocrit, Hemoglobin, Lymphocytes, MCV, Neutrophils, Red Blood Cells.

No microbiology records available.

Average IV fluid/medication input: 87.15 units.

Average fluid output (e.g. urine): 156.46 units.

Patient transferred through units: CCU.

DRG complexity descriptions: CRANIOTOMY AGE >17 EXCEPT FOR TRAUMA.""",
    output_summary="""Female patient admitted on 2194-07-26 for emergency care with a primary diagnosis of brain metastases. The patient was transferred through the Coronary Care Unit (CCU) and had an ICU stay of approximately 2.88 days. DRG complexity corresponds to craniotomy procedures in adults, excluding trauma-related cases.

Diagnoses included secondary malignant neoplasm of the brain and spinal cord, convulsions, and a personal history of uterine cancer. Neurosurgical procedures performed involved excision or destruction of brain tissue and operations on extraocular muscles and tendons, suggesting management of intracranial metastasis.

No medication data was recorded. Abnormal laboratory findings were observed in glucose, albumin, globulin, chloride, and several hematologic indices including hemoglobin, hematocrit, MCV, lymphocytes, neutrophils, and RBC count.

No microbiological cultures were available. The patient had an average IV fluid/medication input of 87.15 units and a fluid output of 156.46 units."""
)

In [73]:
add_manual_entry(
    subject_id=10045,
    target_audience="standard",
    input_summary="""Patient is a F admitted on 2129-11-24 for emergency care with a primary diagnosis of 'FEVER'. Ethnicity: WHITE, Marital Status: MARRIED, Insurance: Medicare. Religion: PROTESTANT QUAKER. Length of ICU stay: 6.3034 days.

Diagnoses include: Septicemia due to anaerobes, Intestinal infection due to Clostridium difficile, Septic shock, Arterial embolism and thrombosis of lower extremity, Atrial fibrillation, Hyposmolality and/or hyponatremia, Acute kidney failure, unspecified, Urinary tract infection, site not specified, Acquired coagulation factor deficiency, Other colostomy and enterostomy complication, Congestive heart failure, unspecified, Intracerebral hemorrhage, Severe sepsis, Stricture of artery, Subendocardial infarction, subsequent episode of care, Unspecified acquired hypothyroidism, Diabetes mellitus without mention of complication, type II or unspecified type, not stated as uncontrolled, Unspecified essential hypertension, Coronary atherosclerosis of native coronary artery, Other and unspecified hyperlipidemia, Anemia, unspecified, Other and unspecified mycoses, Other artificial opening of urinary tract status, Personal history of malignant neoplasm of other parts of uterus, Personal history of irradiation, presenting hazards to health, Personal history of tobacco use, Percutaneous transluminal coronary angioplasty status.

Procedures performed: Other permanent ileostomy, Endarterectomy, lower limb arteries, Angioplasty of other non-coronary vessel(s), Fasciotomy, Continuous intra-arterial blood gas monitoring, Insertion of non-drug-eluting peripheral (non-coronary) vessel stent(s), Procedure on two vessels, Insertion of one vascular stent, Cranial or peripheral nerve graft, Arteriography of femoral and other lower extremity arteries, Parenteral infusion of concentrated nutritional substances, Enteral infusion of concentrated nutritional substances, Transfusion of packed cells, Transfusion of platelets.

Medications prescribed: Multivitamins, Pantoprazole, Ascorbic Acid, Oxycodone-Acetaminophen, Dolasetron Mesylate, Insulin, Amiodarone HCl, sodium bicarb, Aspirin, Metronidazole.

Abnormal lab results observed in: Bicarbonate, Calcium, Total, Creatinine, Glucose, Troponin T, Hematocrit, Hemoglobin, MCHC, RDW, Red Blood Cells.

Microbiology cultures detected organisms: CANDIDA ALBICANS, PRESUMPTIVE IDENTIFICATION, YEAST.

Average IV fluid/medication input: 53.31 units.

Average fluid output (e.g. urine): 66.31 units.

Patient transferred through units: MICU, SICU.

DRG complexity descriptions: Infectious & Parasitic Diseases Including HIV W O.R. Procedure, INFECTIOUS & PARASITIC DISEASES W OR PROCEDURE.""",
    output_summary="""Female patient admitted on 2129-11-24 for emergency care with a primary diagnosis of fever. The patient was transferred through MICU and SICU with a total ICU stay of approximately 6.3 days. DRG complexity describes infectious and parasitic diseases requiring operative procedures.

The clinical course was complicated by septicemia due to anaerobes, septic shock, Clostridium difficile infection, intracerebral hemorrhage, and severe sepsis. Additional comorbidities included atrial fibrillation, acute kidney injury, arterial embolism of the lower limb, urinary tract infection, anemia, coagulopathy, hyponatremia, diabetes mellitus type II, hypothyroidism, and coronary artery disease.

Past medical history included radiation exposure, tobacco use, uterine malignancy, and prior percutaneous coronary intervention.

The patient underwent extensive procedures, including endarterectomy of lower limb arteries, vascular stent placement, cranial or peripheral nerve graft, angioplasty of non-coronary vessels, fasciotomy, arteriography of lower extremity arteries, parenteral and enteral nutritional support, and multiple transfusions.

Medications administered included broad-spectrum antibiotics (Metronidazole), antiplatelets (Aspirin), antiarrhythmics (Amiodarone), insulin, analgesics (Oxycodone-Acetaminophen), proton pump inhibitors (Pantoprazole), sodium bicarbonate, vitamins, and antiemetics (Dolasetron).

Abnormal lab findings were noted in renal function, blood glucose, electrolyte balance (bicarbonate, calcium), cardiac markers (troponin), and several hematologic parameters (RBC, hemoglobin, hematocrit, RDW, MCHC).

Microbiology cultures revealed Candida albicans and other yeast species.

The patient received an average of 53.31 units of IV inputs and had a fluid output of 66.31 units."""
)

In [71]:
add_manual_entry(
    subject_id=10044,
    target_audience="standard",
    input_summary="""Patient is a F admitted on 2152-10-02 for emergency care with a primary diagnosis of 'METASTATIC MELANOMA;BRAIN METASTASIS'. Ethnicity: UNKNOWN/NOT SPECIFIED, Marital Status: UNKNOWN (DEFAULT), Insurance: Medicare. Religion: UNOBTAINABLE. Length of ICU stay: 4.6179 days.

Diagnoses include: Secondary malignant neoplasm of brain and spinal cord, Cellulitis and abscess of leg, except foot, Malignant neoplasm of liver, secondary, Secondary malignant neoplasm of lung, Atrial fibrillation, Unspecified essential hypertension, Coronary atherosclerosis of native coronary artery, Unspecified disorder of kidney and ureter, Anemia in neoplastic disease.

Procedures performed: Other radiotherapeutic procedure, Venous catheterization, not elsewhere classified.

Medications prescribed: Acetaminophen, Dexamethasone, Insulin, Furosemide, Aspirin, Magnesium Sulfate, Hydralazine HCl, Haloperidol, Miconazole Powder 2%, Lorazepam.

Abnormal lab results observed in: Creatinine, Urea Nitrogen, Hematocrit, Hemoglobin, RDW, Red Blood Cells, Ferritin, Potassium, MCHC, Glucose.

No microbiology records available.

Average IV fluid/medication input: 65.4 units.

Average fluid output (e.g. urine): 126.51 units.

Patient transferred through units: MICU.

DRG complexity descriptions: NERVOUS SYSTEM NEOPLASMS WITH COMPLICATIONS, COMORBIDITIES.""",
    output_summary="""Female patient admitted on 2152-10-02 for emergency care with a primary diagnosis of metastatic melanoma with brain metastasis.

The patient was admitted to the MICU with a total ICU stay of 4.62 days. DRG complexity classification indicates a nervous system neoplasm with associated complications and comorbidities.

Significant diagnoses included metastatic spread to the brain, spinal cord, liver, and lung. Additional conditions comprised cellulitis and abscess of the leg, atrial fibrillation, essential hypertension, coronary artery disease, renal dysfunction, and anemia related to malignancy.

The patient underwent venous catheterization and received a radiotherapeutic intervention.

Medications administered included corticosteroids (Dexamethasone), analgesics (Acetaminophen), antihypertensives (Hydralazine), antipsychotics (Haloperidol), antifungals (Miconazole), antiplatelets (Aspirin), diuretics (Furosemide), insulin, magnesium, and anxiolytics (Lorazepam).

Laboratory abnormalities were observed in renal function markers (Creatinine, Urea Nitrogen), hematologic indices (Hematocrit, Hemoglobin, RBC, RDW, MCHC), ferritin, potassium, and glucose levels.

Microbiology data were not available.

The patient had an average IV intake of 65.4 units and a higher-than-intake fluid output of 126.51 units."""
)

In [69]:
add_manual_entry(
    subject_id=10043,
    target_audience="standard",
    input_summary="""Patient is a M admitted on 2185-04-14 for emergency care with a primary diagnosis of 'RESPIRATORY DISTRESS'. Ethnicity: WHITE, Marital Status: nan, Insurance: Medicare. Religion: CATHOLIC. Length of ICU stay: 2.0141 days.

Diagnoses include: Acute respiratory failure, Pneumonia, organism unspecified, Obstructive chronic bronchitis with (acute) exacerbation, Intestinal infection due to Clostridium difficile, Thrombocytopenia, unspecified, Acute diastolic heart failure, Congestive heart failure, unspecified, Anemia of other chronic disease.

Procedures performed: Continuous invasive mechanical ventilation for less than 96 consecutive hours, Insertion of endotracheal tube, Fiber-optic bronchoscopy.

Medications prescribed: Iso-Osmotic Dextrose, NS, D5W, Dextrose 5%, Vial, 1/2 NS, Vancomycin HCl, Norepinephrine, Levofloxacin, Methylprednisolone Na Succ.

Abnormal lab results observed in: Creatine Kinase (CK), Glucose, Troponin T, Urea Nitrogen, Atypical Lymphocytes, Bands, Hematocrit, Hemoglobin, Lymphocytes, Neutrophils.

Microbiology cultures detected organisms: YEAST, CLOSTRIDIUM DIFFICILE.

Average IV fluid/medication input: 132.43 units.

Average fluid output (e.g. urine): 134.79 units.

Patient transferred through units: MICU.

DRG complexity descriptions: RESPIRATORY SYSTEM DIAGNOSIS WITH VENTILATOR SUPPORT.""",
    output_summary="""Male patient admitted on 2185-04-14 for emergency care with a primary diagnosis of respiratory distress.

The patient was admitted to the MICU for 2.01 days. DRG classification corresponds to a respiratory system diagnosis requiring ventilator support.

The clinical picture was dominated by acute respiratory failure, pneumonia (unspecified organism), and obstructive chronic bronchitis with acute exacerbation. Additional systemic conditions included C. difficile intestinal infection, thrombocytopenia, acute diastolic heart failure, chronic congestive heart failure, and anemia of chronic disease.

Key interventions included mechanical ventilation lasting less than 96 hours, endotracheal intubation, and fiber-optic bronchoscopy.

Pharmacological management involved antibiotics (Vancomycin, Levofloxacin), corticosteroids (Methylprednisolone), vasopressors (Norepinephrine), and a range of IV fluids (Dextrose solutions, NS, ½ NS).

Abnormal laboratory results indicated elevations in CK, glucose, urea nitrogen, and Troponin T. Hematological changes included altered lymphocyte and neutrophil counts, atypical lymphocytes, bands, low hematocrit and hemoglobin levels.

Microbiological testing revealed presence of Clostridium difficile and yeast organisms.

Fluid data showed a balanced profile with an input of 132.43 units and output of 134.79 units."""
)

In [67]:
add_manual_entry(
    subject_id=10042,
    target_audience="standard",
    input_summary="""Patient is a M admitted on 2147-02-06 for emergency care with a primary diagnosis of 'UNSTABLE ANGINA'. Ethnicity: WHITE, Marital Status: nan, Insurance: Medicare. Religion: UNOBTAINABLE. Length of ICU stay: 8.1415 days.

Diagnoses include: Subendocardial infarction, initial episode of care, Cardiac complications, not elsewhere classified, Paroxysmal ventricular tachycardia, Congestive heart failure, unspecified, Mitral valve disorders, Atrial fibrillation, Primary pulmonary hypertension, Coronary atherosclerosis of native coronary artery, Peripheral vascular disease, unspecified.

Procedures performed: (Aorto)coronary bypass of two coronary arteries, Single internal mammary-coronary artery bypass, Extracorporeal circulation auxiliary to open heart surgery, Implant of pulsation balloon, Arterial catheterization, Diagnostic ultrasound of heart, Catheter based invasive electrophysiologic testing, Transfusion of packed cells.

Medications prescribed: Heparin Sodium, Nitroglycerin, Potassium Chloride, Nitroprusside Sodium, Phenylephrine HCl, Vancomycin HCl, Humulin-R Insulin, Norepinephrine, Milrinone, Atorvastatin.

Abnormal lab results observed in: Alkaline Phosphatase, Asparate Aminotransferase (AST), Bilirubin, Direct, Bilirubin, Total, Lactate Dehydrogenase (LD), Troponin T, Hematocrit, Hemoglobin, MCH, PT.

No microbiology records available.

Average IV fluid/medication input: 173.9 units.

Average fluid output (e.g. urine): 110.66 units.

Patient transferred through units: CSRU.

DRG complexity descriptions: CORONARY BYPASS WITHOUT CARDIAC CATHETER.""",
    output_summary="""Male patient admitted on 2147-02-06 for emergency care with a primary diagnosis of unstable angina.

ICU stay spanned 8.14 days in the CSRU. DRG classification denotes coronary artery bypass surgery performed without concurrent cardiac catheterization.

The patient had significant cardiac comorbidities including subendocardial infarction, ventricular tachycardia, atrial fibrillation, mitral valve disease, heart failure, coronary artery disease, and primary pulmonary hypertension. Peripheral vascular disease was also noted.

A major cardiovascular surgical intervention was performed, including aorto-coronary bypass of two arteries, use of a single internal mammary artery graft, and extracorporeal circulation. Additional procedures included arterial catheterization, electrophysiologic testing, diagnostic echocardiography, intra-aortic balloon pump placement, and packed red cell transfusion.

Medications included vasodilators and anti-anginal agents (Nitroglycerin, Nitroprusside), inotropes/vasopressors (Norepinephrine, Milrinone, Phenylephrine), anticoagulants (Heparin), insulin (Humulin-R), antibiotics (Vancomycin), and lipid-lowering agents (Atorvastatin). Electrolyte support was provided via Potassium Chloride.

Abnormal labs showed evidence of hepatic dysfunction (elevated bilirubin, AST, ALP, LD), myocardial injury (elevated Troponin T), and hematologic abnormalities (low hematocrit, hemoglobin, altered MCH and PT).

No microbiological organisms were identified during the ICU stay.

Fluid data showed a high input of 173.9 units with a slightly lower output of 110.66 units."""
)

In [65]:
add_manual_entry(
    subject_id=10040,
    target_audience="standard",
    input_summary="""Patient is a F admitted on 2147-02-23 for emergency care with a primary diagnosis of 'PULMONARY EDEMA\\CATH'. Ethnicity: UNKNOWN/NOT SPECIFIED, Marital Status: nan, Insurance: Medicare. Religion: UNOBTAINABLE. Length of ICU stay: 2.1306 days.

Diagnoses include: Unspecified hypertensive heart disease with heart failure, Bacteremia, Other emphysema, Acute respiratory failure, Aortic valve disorders, Congestive heart failure, unspecified, Coronary atherosclerosis of native coronary artery, Iron deficiency anemia, unspecified, Esophageal reflux.

Procedures performed: Insertion of drug-eluting coronary artery stent(s), Combined right and left heart cardiac catheterization, Angiocardiography of right heart structures, Coronary arteriography using two catheters, Injection or infusion of platelet inhibitor, Continuous invasive mechanical ventilation for less than 96 consecutive hours, Insertion of endotracheal tube.

Medications prescribed: Furosemide, Ferrous Sulfate, Multivitamins, Pantoprazole, Potassium Chloride, Sodium Chloride 0.9%  Flush, Lisinopril, Pneumococcal Vac Polyvalent, Losartan Potassium, Hydrochlorothiazide.

Abnormal lab results observed in: Albumin, Asparate Aminotransferase (AST), Chloride, Cholesterol, Total, Troponin T, Urea Nitrogen, Hematocrit, Hemoglobin, Lymphocytes, MCH.

Microbiology cultures detected organisms: STREPTOCOCCUS PNEUMONIAE.

Average IV fluid/medication input: 45.99 units.

Average fluid output (e.g. urine): 139.7 units.

Patient transferred through units: MICU.

DRG complexity descriptions: PERCUTANEOUS CARDIOVASCULAR PROCEDURE WITH DRUG-ELUDING STENT WITHOUT ACUTE MYOCARDIAL INFARCTION.""",
    output_summary="""Female patient admitted on 2147-02-23 for emergency care with a primary diagnosis of pulmonary edema requiring catheterization.

ICU stay lasted 2.13 days in the MICU. DRG classification reflects a percutaneous cardiovascular procedure with drug-eluting stent placement, without acute myocardial infarction.

Major diagnoses included hypertensive heart disease with heart failure, acute respiratory failure, bacteremia, emphysema, aortic valve disorders, and coronary artery disease. Additional conditions included iron deficiency anemia, esophageal reflux, and unspecified congestive heart failure.

Invasive cardiovascular and respiratory procedures performed included right and left heart catheterization, angiocardiography, coronary arteriography, stent placement, mechanical ventilation (<96 hrs), and endotracheal intubation.

Medications addressed heart failure and hypertension (Furosemide, Lisinopril, Losartan, Hydrochlorothiazide), anemia (Ferrous Sulfate), gastric protection (Pantoprazole), electrolyte balance (Potassium Chloride), general supplementation (Multivitamins), and infection prevention (Pneumococcal Vaccine).

Abnormal labs were noted in liver function (AST), renal function (Urea Nitrogen), cardiac markers (Troponin T), electrolytes (Chloride, Albumin), lipids (Cholesterol), and hematology (Hematocrit, Hemoglobin, Lymphocytes, MCH).

Streptococcus pneumoniae was identified via microbiology cultures.

Fluid management data showed an average input of 45.99 units and significantly higher output at 139.7 units."""
)

In [63]:
add_manual_entry(
    subject_id=10038,
    target_audience="standard",
    input_summary="""Patient is a F admitted on 2144-02-09 for emergency care with a primary diagnosis of 'FAILURE TO THRIVE'. Ethnicity: WHITE, Marital Status: WIDOWED, Insurance: Medicare. Religion: CATHOLIC. Length of ICU stay: 3.3857 days.

Diagnoses include: Infection and inflammatory reaction due to other internal orthopedic device, implant, and graft, Streptococcal septicemia, Atrial fibrillation, Congestive heart failure, unspecified, Acute kidney failure, unspecified, Sepsis, Diastolic heart failure, unspecified, Pyogenic arthritis, pelvic region and thigh, Alkalosis, Thrombocytopenia, unspecified, Other complications due to internal joint prosthesis, Adult failure to thrive, Unspecified essential hypertension, Other persistent mental disorders due to conditions classified elsewhere, Venous (peripheral) insufficiency, unspecified, Surgical operation with implant of artificial internal device causing abnormal patient reaction, or later complication,without mention of misadventure at time of operation, Accidents occurring in other specified places, Osteoporosis, unspecified, Abnormal coagulation profile, Streptococcus infection in conditions classified elsewhere and of unspecified site, streptococcus, group B.

Procedures performed: Arthrocentesis, Diagnostic ultrasound of heart, Venous catheterization, not elsewhere classified.

Medications prescribed: Lorazepam, Sodium Chloride 0.9%  Flush, Heparin, Phytonadione, Potassium Chloride, Diltiazem, Morphine Sulfate, Insulin, Heparin Flush PICC (100 units/ml), Neutra-Phos.

Abnormal lab results observed in: Creatinine, Glucose, NTproBNP, Troponin T, Urea Nitrogen, Hematocrit, INR(PT), Lymphocytes, Neutrophils, Platelet Count.

Microbiology cultures detected organisms: BETA STREPTOCOCCUS GROUP B.

Average IV fluid/medication input: 123.66 units.

Average fluid output (e.g. urine): 44.69 units.

Patient transferred through units: SICU.

DRG complexity descriptions: Malfunction, Reaction & Comp of Orthopedic Device or Procedure, AFTERCARE, MUSCULOSKELETAL SYSTEM & CONNECTIVE TISSUE.""",
    output_summary="""Female patient admitted on 2144-02-09 for emergency care with a primary diagnosis of failure to thrive.

ICU stay lasted approximately 3.39 days in the SICU. DRG classification indicates complications and aftercare related to orthopedic devices within the musculoskeletal system.

Significant diagnoses included sepsis, streptococcal septicemia (Group B), pyogenic arthritis of the pelvic region, acute kidney failure, atrial fibrillation, and both congestive and diastolic heart failure. Chronic and procedural complications involved infections and inflammatory reactions due to orthopedic implants, thrombocytopenia, osteoporosis, hypertension, venous insufficiency, and abnormal coagulation.

Procedures performed included arthrocentesis, diagnostic echocardiography, and venous catheterization.

Medications addressed infection and inflammation (Heparin, Phytonadione), sedation (Lorazepam), pain (Morphine Sulfate), cardiovascular issues (Diltiazem), electrolyte support (Potassium Chloride, Neutra-Phos), and glycemic control (Insulin). Sodium chloride and Heparin flushes were also used for line maintenance.

Abnormal lab findings indicated renal dysfunction (Creatinine, Urea Nitrogen), cardiac stress (NTproBNP, Troponin T), glucose imbalance, hematologic abnormalities (Hematocrit, Platelet Count), and immune cell shifts (Lymphocytes, Neutrophils). Coagulation profile (INR) was also abnormal.

Microbiology cultures confirmed infection with beta-hemolytic Streptococcus Group B.

Fluid management showed average IV input of 123.66 units and comparatively low output at 44.69 units."""
)

In [61]:
add_manual_entry(
    subject_id=10036,
    target_audience="standard",
    input_summary="""Patient is a F admitted on 2185-03-24 for emergency care with a primary diagnosis of 'SEPSIS'. Ethnicity: WHITE, Marital Status: MARRIED, Insurance: Medicare. Religion: JEWISH. Length of ICU stay: 1.8068 days.

Diagnoses include: Unspecified septicemia, Septic shock, Pneumonitis due to inhalation of food or vomitus, Acute kidney failure, unspecified, Acidosis, Hyperosmolality and/or hypernatremia, Severe sepsis, Esophageal reflux, Alzheimer's disease, Dementia in conditions classified elsewhere without behavioral disturbance, Polymyalgia rheumatica, Unspecified acquired hypothyroidism.

Procedures performed: Venous catheterization, not elsewhere classified.

Medications prescribed: Cosyntropin, Bisacodyl, Docusate Sodium, Heparin Flush CVL  (100 units/ml), Heparin, Morphine Sulfate, Insulin, Aspirin, Diazepam, Norepinephrine.

Abnormal lab results observed in: Asparate Aminotransferase (AST), Bicarbonate, Calcium, Total, Chloride, Creatinine, Glucose, Lipase, Magnesium, Sodium, Troponin T.

No microbiology records available.

Average IV fluid/medication input: 67.26 units.

Average fluid output (e.g. urine): 82.37 units.

Patient transferred through units: MICU.

DRG complexity descriptions: SEPTICEMIA AGE >17.""",
    output_summary="""Female patient admitted on 2185-03-24 for emergency care with a primary diagnosis of sepsis.

ICU stay lasted 1.81 days in the MICU. DRG classification indicates septicemia in patients over 17 years.

Notable comorbidities included septic shock, aspiration pneumonitis, acute kidney failure, metabolic derangements (acidosis, hypernatremia/hyperosmolality), and severe sepsis. Chronic conditions included esophageal reflux, Alzheimer's disease, dementia, hypothyroidism, and polymyalgia rheumatica.

One invasive procedure was recorded: venous catheterization.

Medication regimen addressed infection (Norepinephrine), hypothyroid and gastrointestinal support (Heparin, Docusate, Bisacodyl), endocrine diagnostics (Cosyntropin), analgesia (Morphine), anticoagulation (Aspirin, Heparin), glycemic control (Insulin), and sedation (Diazepam).

Abnormal labs reflected renal impairment (Creatinine), liver involvement (AST), electrolyte imbalances (Bicarbonate, Calcium, Chloride, Sodium, Magnesium), and elevated Troponin T and Glucose.

Average IV fluid/medication input was 67.26 units, with fluid output of 82.37 units."""
)


In [59]:
add_manual_entry(
    subject_id=10035,
    target_audience="standard",
    input_summary="""Patient is a M admitted on 2129-03-03 for elective care with a primary diagnosis of 'RECURRENT LEFT CAROTID STENOSIS,PRE HYDRATION'. Ethnicity: WHITE, Marital Status: MARRIED, Insurance: Medicare. Religion: NOT SPECIFIED. Length of ICU stay: 1.3755 days.

Diagnoses include: Occlusion and stenosis of carotid artery without mention of cerebral infarction, Gout, unspecified, Old myocardial infarction.

Procedures performed: Angioplasty of other non-coronary vessel(s), Insertion of non-drug-eluting peripheral (non-coronary) vessel stent(s).

Medications prescribed: Heparin Sodium, Nitroprusside Sodium, Potassium Chloride, D5W, SW, Clopidogrel Bisulfate, Acetylcysteine 20%, Diazepam, Nitroglycerin SL, Pravastatin.

Abnormal lab results observed in: Creatinine, Glucose, Hemoglobin, Red Blood Cells, Hematocrit, PT, PTT, Magnesium, Free Calcium.

No microbiology records available.

Average IV fluid/medication input: 939.91 units.

Average fluid output (e.g. urine): 125.65 units.

Patient transferred through units: SICU.

DRG complexity descriptions: EXTRACRANIAL VASCULAR PROCEDURES.""",
    output_summary="""Male patient admitted on 2129-03-03 for elective care due to recurrent left carotid artery stenosis, with pre-hydration noted as part of the care plan.

Comorbidities included occlusion and stenosis of the carotid artery without infarction, gout, and a history of myocardial infarction.

The patient underwent angioplasty and placement of non-drug-eluting stents in peripheral (non-coronary) vessels.

Pharmacologic management included antithrombotics (Heparin Sodium, Clopidogrel Bisulfate), vasodilators (Nitroprusside Sodium, Nitroglycerin SL), statins (Pravastatin), electrolyte and fluid therapy (Potassium Chloride, D5W, SW), antioxidant support (Acetylcysteine), and adjunctive sedation (Diazepam).

Abnormal labs included renal (Creatinine), glycemic (Glucose), hematologic (Hemoglobin, RBC, Hematocrit), and coagulation parameters (PT, PTT), along with electrolyte disturbances in magnesium and calcium.

ICU stay was 1.38 days in the SICU. The patient received a notably high volume of IV fluids/medications (939.91 units), with fluid output recorded at 125.65 units.

DRG classification: Extracranial vascular procedures."""
)

In [98]:
import json

def convert_manual_dataset_to_jsonl(dataset, output_file="healthcare_finetune_dataset.jsonl"):
    """
    Converts the manually created 'manual_dataset' into a chat-style JSONL dataset
    suitable for LLM fine-tuning with instruction-following format.

    Args:
        dataset (List[Dict]): Each dict contains 'subject_id', 'target_audience', 'input', 'output'.
        output_file (str): Output .jsonl file path.
    """
    with open(output_file, "w", encoding="utf-8") as f:
        for entry in dataset:
            user_prompt = (
                f"You are a clinical language model. Generate a concise and professional summary of the patient's ICU stay "
                f"based on the structured information provided below for the intended audience: {entry['target_audience']}.\n\n"
                f"Structured Data:\n{entry['input']}"
            )
            record = {
                "messages": [
                    {"role": "system", "content": "You are a clinical language model trained to summarize ICU patient records."},
                    {"role": "user", "content": user_prompt.strip()},
                    {"role": "assistant", "content": entry["output"].strip()}
                ]
            }
            f.write(json.dumps(record) + "\n")

    print(f"✅ Saved {len(dataset)} entries to '{output_file}'")

In [99]:
convert_manual_dataset_to_jsonl(manual_dataset, "finetune_data_healthcare.jsonl")

✅ Saved 30 entries to 'finetune_data_healthcare.jsonl'


In [102]:
pip install datasets peft accelerate bitsandbytes


In [112]:
pip install peft trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.6/504.6 kB 9.8 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="finetune_data_healthcare.jsonl", split="train")

In [4]:
def format_chat(example):
    text = ""
    for msg in example["messages"]:
        role = msg["role"]
        content = msg["content"]
        if role == "system":
            text += f"<|system|>\n{content}\n"
        elif role == "user":
            text += f"<|user|>\n{content}\n"
        elif role == "assistant":
            text += f"<|assistant|>\n{content}\n"
    return {"text": text.strip()}

dataset = dataset.map(format_chat)

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [115]:
formatted_dataset.to_json("formatted_healthcare_finetune_data.jsonl")
print("✅ Saved formatted dataset for training.")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

✅ Saved formatted dataset for training.


In [120]:
!pip install -U bitsandbytes

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=1024,
    )

tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [7]:
!pip install -U peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.16.0
    Uninstalling peft-0.16.0:
      Successfully uninstalled peft-0.16.0


In [3]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

# Prepare quantized model for LoRA
model = prepare_model_for_kbit_training(model)

# Define LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # for llama-type models
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Attach LoRA adapters
model = get_peft_model(model, lora_config)

In [5]:
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling

# Load the JSONL dataset you created earlier
dataset = load_dataset("json", data_files="finetune_data_healthcare.jsonl")

# Function to tokenize each prompt-response pair
def tokenize_function(example):
    full_text = "".join([msg["content"] for msg in example["messages"]])
    return tokenizer(full_text, truncation=True, padding="max_length", max_length=512)

# Apply tokenization
tokenized_dataset = dataset["train"].map(tokenize_function, batched=False)

# Data collator for Causal Language Modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [6]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./tinyllama-healthcare-finetuned",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
    fp16=True,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [7]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.239900
20,2.192300


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.239900
20,2.192300
30,2.106200
40,2.086400


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=45, training_loss=2.141932551066081, metrics={'train_runtime': 8445.2183, 'train_samples_per_second': 0.011, 'train_steps_per_second': 0.005, 'total_flos': 286644838072320.0, 'train_loss': 2.141932551066081, 'epoch': 3.0})

In [8]:
output_dir = "finetuned_tinyllama"
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('finetuned_tinyllama/tokenizer_config.json',
 'finetuned_tinyllama/special_tokens_map.json',
 'finetuned_tinyllama/chat_template.jinja',
 'finetuned_tinyllama/tokenizer.model',
 'finetuned_tinyllama/added_tokens.json',
 'finetuned_tinyllama/tokenizer.json')

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model = AutoModelForCausalLM.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cpu


In [10]:
prompt = "Summarize the patient's condition in simple terms for their family:"
response = pipe(prompt, max_new_tokens=150, do_sample=True, top_k=50, top_p=0.95)
print(response[0]['generated_text'])

Summarize the patient's condition in simple terms for their family:

The patient's condition is complicated by severe inflammation of the brain stem and spinal cord, with extensive brain tissue damage and impairment of cognitive function. They require intensive care and medical attention, including physical and occupational therapies, speech and language therapies, and specialized diagnostic tests to monitor their progress. The family can expect to be there for the patient through the difficult and often emotionally challenging journey ahead.


In [14]:
input_text = ("""Patient is a M admitted on 2129-03-03 for elective care with a primary diagnosis of 'RECURRENT LEFT CAROTID STENOSIS,PRE HYDRATION'. Ethnicity: WHITE, Marital Status: MARRIED, Insurance: Medicare. Religion: NOT SPECIFIED. Length of ICU stay: 1.3755 days.

Diagnoses include: Occlusion and stenosis of carotid artery without mention of cerebral infarction, Gout, unspecified, Old myocardial infarction.

Procedures performed: Angioplasty of other non-coronary vessel(s), Insertion of non-drug-eluting peripheral (non-coronary) vessel stent(s).

Medications prescribed: Heparin Sodium, Nitroprusside Sodium, Potassium Chloride, D5W, SW, Clopidogrel Bisulfate, Acetylcysteine 20%, Diazepam, Nitroglycerin SL, Pravastatin.

Abnormal lab results observed in: Creatinine, Glucose, Hemoglobin, Red Blood Cells, Hematocrit, PT, PTT, Magnesium, Free Calcium.

No microbiology records available.

Average IV fluid/medication input: 939.91 units.

Average fluid output (e.g. urine): 125.65 units.

Patient transferred through units: SICU.

DRG complexity descriptions: EXTRACRANIAL VASCULAR PROCEDURES.

Generate the report for this patient:"""

)
response = pipe(input_text, max_new_tokens=150, do_sample=True, top_p=0.95)
print(response[0]["generated_text"])

Patient is a M admitted on 2129-03-03 for elective care with a primary diagnosis of 'RECURRENT LEFT CAROTID STENOSIS,PRE HYDRATION'. Ethnicity: WHITE, Marital Status: MARRIED, Insurance: Medicare. Religion: NOT SPECIFIED. Length of ICU stay: 1.3755 days.

Diagnoses include: Occlusion and stenosis of carotid artery without mention of cerebral infarction, Gout, unspecified, Old myocardial infarction.

Procedures performed: Angioplasty of other non-coronary vessel(s), Insertion of non-drug-eluting peripheral (non-coronary) vessel stent(s).

Medications prescribed: Heparin Sodium, Nitroprusside Sodium, Potassium Chloride, D5W, SW, Clopidogrel Bisulfate, Acetylcysteine 20%, Diazepam, Nitroglycerin SL, Pravastatin.

Abnormal lab results observed in: Creatinine, Glucose, Hemoglobin, Red Blood Cells, Hematocrit, PT, PTT, Magnesium, Free Calcium.

No microbiology records available.

Average IV fluid/medication input: 939.91 units.

Average fluid output (e.g. urine): 125.65 units.

Patient trans